In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import numpy as np
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

path_to_catalog=r'C:\Users\kgrzebien\Desktop\python_tasks\kupony\fast_kpi\\'

# preprocessing

In [2]:
def func_str_to_float(s):
    s=s.astype(str)
    s=s.apply(lambda x: x.replace(',','.'))
    s=s.astype(float)
    return s



def func_date(df, min_date):
    df['date_start']=df['Z_DNIA']+' '+df['CZAS_POCZ']
    df['date_start']=pd.to_datetime(df['date_start'], format='%y/%m/%d %H:%M:%S')
    df['date_start']=df['date_start'].dt.floor('Min')
    df['date_finish']=df['Z_DNIA']+' '+df['CZAS_KON']
    df['date_finish']=pd.to_datetime(df['date_finish'], format='%y/%m/%d %H:%M:%S')
    df['date_finish']=df['date_finish'].dt.floor('Min')
    df['Z_DNIA']=pd.to_datetime(df['Z_DNIA'], format='%y/%m/%d')
    
    df=df.loc[df['Z_DNIA']>=np.datetime64(min_date)].reset_index().drop('index', axis=1)
    df['year']=df['Z_DNIA'].apply(lambda x: x.year)
    df['month']=df['Z_DNIA'].apply(lambda x: x.month)
    df['day_of_year']=df['Z_DNIA'].apply(lambda x: x.timetuple().tm_yday)
    df['week']=df['Z_DNIA'].apply(lambda x: x.isocalendar()[1])
    df['week_day']=df['Z_DNIA'].apply(lambda x: x.isocalendar()[2])
    df['year_month']=df['year'].astype(str)+'-'+df['month'].astype(str)+'-'+'01'
    df['year_month']=df['year_month'].apply(lambda x: x[:5]+'0'+x[5:] if len(x)==9 else x)
    df['year_month']=pd.to_datetime(df['year_month'], format='%Y-%m-%d')
    df['year_week']=df['year'].astype(str)+'--'+df['week'].astype(str)
    df['year_week']=df['year_week'].apply(lambda x: x[:6]+'0'+x[6:] if len(x)==7 else x)
    df['year_day']=df['year'].astype(str)+'--'+df['day_of_year'].astype(str)
    df['year_day']=df['year_day'].apply(lambda x: x[:6]+'0'+x[6:] if len(x)<9 else x)
    
    for i in ['CZAS_POCZ','CZAS_KON']:
        df[i+'_hour']=df[i].str.split(':')
        df[i+'_hour']=df[i+'_hour'].apply(lambda x: x[0])
        df[i+'_minutes']=df[i].str.split(':')
        df[i+'_minutes']=df[i+'_minutes'].apply(lambda x: x[1])
        df[i+'_hour_minutes']=df[i+'_hour']+':'+df[i+'_minutes']
        df[i+'_hour']=df[i+'_hour'].astype(int)
        df[i+'_minutes']=df[i+'_minutes'].astype(int)
        
    df=df.loc[df['CZAS_POCZ_hour']>=4].loc[df['CZAS_POCZ_hour']<=23].loc[df['CZAS_KON_hour']>=5].loc[df['CZAS_KON_hour']<=24].reset_index().drop('index', axis=1)
    #df['zmiana']=df['CZAS_KON'].apply(lambda x: 'pierwsza' if x<='14:00' else 'druga')
    return df

def func_add_name_workers(df, name_of_file_with_workers):
    df_pracownicy=pd.read_excel(name_of_file_with_workers)
    df_pracownicy['NUMER']=df_pracownicy['NUMER']#.astype(str)
    df=df.merge(df_pracownicy, how='left', left_on='TNACA1', right_on='NUMER')
    df=df.drop('NUMER', axis=1)
    df=df.rename(columns={'PELNANAZWA':'TNACA1_NAZWISKO'})
    df=df.merge(df_pracownicy, how='left', left_on='TNACA2', right_on='NUMER')
    df=df.drop('NUMER', axis=1)
    df=df.rename(columns={'PELNANAZWA':'TNACA2_NAZWISKO'})

    df['TNACA1_NAZWISKO']=df['TNACA1_NAZWISKO'].apply(lambda x: x.split(' ')[0][0]+'. '+x.split(' ')[1] if x!='NIEZNANY' else x)
    df['TNACA2_NAZWISKO']=df['TNACA2_NAZWISKO'].apply(lambda x: x.split(' ')[0][0]+'. '+x.split(' ')[1] if x!='NIEZNANY' else x)
    return df

def func_create_table(df):
    df['STOL']=df['TNACA1'].astype(str)+'-'+df['TNACA2'].astype(str)
    df['STOL_NAZWISKO']=df['TNACA1_NAZWISKO'].astype(str)+'-'+df['TNACA2_NAZWISKO'].astype(str)
    return df

def func_name_of_column_with_tnaca(name, ktora_tnaca_nazwisko):
    if ktora_tnaca_nazwisko=='TNACA1_NAZWISKO':
        name=name+'tnaca1'
    else: 
        name=name+'tnaca2'
    return name

def func_add_first_and_last_date_workers_and_unique_days(df, ktora_tnaca_nazwisko):
    for i in ['min','max']:
        name=i+'_dzien_pracy_'
        name=func_name_of_column_with_tnaca(name, ktora_tnaca_nazwisko)
        
        _=df.groupby([ktora_tnaca_nazwisko]).agg({'Z_DNIA':i}).reset_index()
        _=_.rename(columns={'Z_DNIA':name})
        df=df.merge(_, how='left', on=ktora_tnaca_nazwisko)
        
    df_unikatowe_dni_tnaca=df.groupby(ktora_tnaca_nazwisko).\
                            agg(liczba_unikatowych_dni_pracy=('Z_DNIA','nunique'),
                                unikatowe_dni_pracy=('Z_DNIA','unique')).reset_index()    
    
    df_unikatowe_dni_tnaca=df_unikatowe_dni_tnaca.rename(columns={'liczba_unikatowych_dni_pracy':func_name_of_column_with_tnaca('liczba_unikatowych_dni_pracy_',ktora_tnaca_nazwisko),
                                                                  'unikatowe_dni_pracy':func_name_of_column_with_tnaca('unikatowe_dni_pracy_',ktora_tnaca_nazwisko)})
    
    df=df.merge(df_unikatowe_dni_tnaca[[ktora_tnaca_nazwisko, func_name_of_column_with_tnaca('liczba_unikatowych_dni_pracy_',ktora_tnaca_nazwisko)]], 
                how='left', on=ktora_tnaca_nazwisko)
    return df, df_unikatowe_dni_tnaca


def func_time_for_fv(df):
    df_tmp2=df.groupby(list(set(df.columns)-set(['NAZWA', 'RODZAJ', 'KOD_TEMA', 'ILOSC']))).\
                agg({'KOD_TEMA':'count',
                     'ILOSC':'sum'}).reset_index()
    
    df_tmp2['time']=df_tmp2['date_finish']-df_tmp2['date_start']
    df_tmp2=df_tmp2.sort_values(by='time').reset_index().drop('index',axis=1)
    df_tmp2['time']=df_tmp2['time']/np.timedelta64(1,'s')/60
    df_tmp2=df_tmp2.loc[df_tmp2['time'].notnull()]
    return df_tmp2
    
    
def func_add_rozmiarsi(df, path_to_slownikid):
    df_slownik=pd.read_csv(path_to_slownikid, #'../../słowniki/slownikid.csv'
                               encoding='Cp1250',
                                sep="|",
                              low_memory=False)
    df_kod_tema_with_rozmiar_si=df_slownik.merge(df[['KOD_TEMA']].drop_duplicates(), how='inner', on='KOD_TEMA')[['ROZMIARSI','KOD_TEMA']].drop_duplicates()
    
    def func_rozmiar_si(str_x):
        if str_x=='':
            return 0

        if 'cm' in str_x:
            return str_x.split('cm')[0]
        elif str_x[0]=='0':
            return str(float(str_x.replace(',','.'))*100)
        else: return str_x
    df_kod_tema_with_rozmiar_si['ROZMIARSI_short']=df_kod_tema_with_rozmiar_si['ROZMIARSI'].apply(lambda x: x.split(' ')[1] if x[0]=='s' else (x.split(' ')[0]))
    df_kod_tema_with_rozmiar_si['ROZMIARSI_short']=df_kod_tema_with_rozmiar_si['ROZMIARSI_short'].apply(lambda x: func_rozmiar_si(x))
    df_kod_tema_with_rozmiar_si['ROZMIARSI_short']=df_kod_tema_with_rozmiar_si['ROZMIARSI_short'].astype(float)
    df=df.merge(df_kod_tema_with_rozmiar_si[['KOD_TEMA','ROZMIARSI_short']], how='left', on='KOD_TEMA')
    df=df.rename(columns={'ROZMIARSI_short':'ROZMIARSI'})
    return df


def func_name_workers_filia(list_of_workers,
                            name_of_file_with_workers):
    
    df=pd.read_excel(name_of_file_with_workers)
    df=df.rename(columns={'PELNANAZWA':'pracownik_nazwisko'})
    df['pracownik_nazwisko']=df['pracownik_nazwisko'].apply(lambda x: x.split(' ')[0][0]+'. '+x.split(' ')[1] if len(x.split(' '))>1 else x)
    df=df.loc[df['pracownik_nazwisko'].isin(list_of_workers)]
    return df.reset_index().drop('index', axis=1)

# read & download

In [3]:
def func_append_csv(list_of_name, col_data, katalog):
    """FUNKCJA
        -dodajaca csv
        -usuwa ostatnie dni, bo mogą być niepełne
    """
    flaga=1
    for file in list_of_name:
        if flaga:
            DF=pd.read_csv(katalog+file,
                          encoding='Cp1250',
                          sep="|")
            try:
                DF=DF.loc[DF['IS_DELETED']=='N']
            except: pass
            DF=DF.loc[DF[col_data]<DF[col_data].max()]
            flaga=0
            print(DF[col_data].max())
        else:
            df=pd.read_csv(katalog+file,
                          encoding='Cp1250',
                          sep="|")
            try:
                df=df.loc[df['IS_DELETED']=='N']
            except: pass
            df=df.loc[df[col_data]<df[col_data].max()]
            df=df.loc[df[col_data]>DF[col_data].max()]
            DF=DF.append(df, ignore_index=True)
            print(DF[col_data].max())
            
    return DF

In [4]:
def func_append_last_n13(df_old, df_new, col_data):
    """FUNKCJA
        -dodajaca pobrane dane do zapisanych
        -usuwa ostatni dzień
    """
    df_new=df_new.loc[df_new[col_data]<df_new[col_data].max()]
    df_old=df_old.append(df_new, ignore_index=True)
    return df_old

In [5]:
def func_query(jakie_zapytanie, date):
    """FUNKCJA
        -podaje zapytanie, które jest potrzebne
    """
    if jakie_zapytanie=='n13':
        query="""
                with "faktury_po_n13_mm" as (

                select "zamow1"."FAKTURA",
                        "n13"."WARTOSC_B",
                        "n13"."WARTOSC_N",
                        "n13"."Z_DNIA"
                from "HURT00"."F60A_ZAMOW1" "zamow1"

                inner join "HURT00"."F60N13" "n13"
                on "zamow1"."FAKTURA_KD" = "n13"."DOKUMENT"

                where  "zamow1"."IS_DELETED"='N'  and "n13"."IS_DELETED"='N'
                    and "zamow1"."FAKTURA_KD"<>' ' and "n13"."FAKTURA_KD"<>' '
                )

                select  "zamow1"."FAKTURA",
                        "zamow1"."DOKUMENT",
                        "zamow1"."FAKTURA_KD",
                        "faktury_po_n13_mm"."Z_DNIA",
                        "zamow1"."TYP_DOKUM",
                        "zamow1"."CZAS_POCZ",
                        "zamow1"."CZAS_KON",
                        "zamow1"."CZAS_ZAPIS",
                        "zamow2"."NAZWA",
                        "zamow2"."RODZAJ",
                        "zamow2"."KOD_TEMA",
                        "zamow2"."ILOSC",
                        "zamow1"."TNACA1",
                        "zamow1"."TNACA2",
                        "faktury_po_n13_mm"."WARTOSC_B",
                        "faktury_po_n13_mm"."WARTOSC_N"
                from "HURT00"."F60A_ZAMOW1" "zamow1"

                inner join "faktury_po_n13_mm" 
                on "faktury_po_n13_mm"."FAKTURA"="zamow1"."FAKTURA"

                left join "HURT00"."F60A_ZAMOW2" "zamow2"
                on "zamow1"."FAKTURA" = "zamow2"."FAKTURA"


                where "zamow1"."Z_DNIA">'{}' --and "Z_DNIA"<'22/01/01'
                    and "zamow1"."IS_DELETED"='N' and "zamow2"."IS_DELETED"='N'
    
                """
        return query.format(date)
    elif jakie_zapytanie=='rcp':
        query="""
                with aaa as(
                            select
                                    --substr(extract(year from DATA),3,4) new_year,
                                    --extract(month from DATA) month,
                                    --extract(day from DATA) day,
                                    to_date('20' || substr(extract(year from DATA),3,4) || '/' || extract(month from DATA) || '/' || extract(day from DATA), 'YYYY/MM/DD') new_date,
                                    IS_DELETED,
                                    ID_ZAD,
                                    ID_ZAD_PG,
                                    ID_ZAD_IND,
                                    ID_ZAD_DOD,
                                    DATA,
                                    ROZ_DNIA,
                                    ZAK_DNIA,
                                    GODZ_PL,
                                    GODZ_ROZ,
                                    GODZ_ZAK,
                                    KOD_OPER,
                                    KOD_GR_RCP,
                                    ID_DOK,
                                    NR_DOK,
                                    ID_POZ_DOK,
                                    OP_POZ_DOK
                                FROM HURT00.EWID_RCP
                                where IS_DELETED='N'
                                )
                select * 
                from aaa
                where new_date>='{}' and new_date<='{}' 
                """
        return query.format(date, datetime.today().strftime("%Y/%m/%d"))
    else:
        return

In [6]:
def func_save_last_n13(df):
    """FUNKCJA
        -zapisuje nowo pobrane dane z n13
    """
    list_of_names=func_read_list_of_names(r'C:\Users\kgrzebien\Desktop\python_tasks\kupony'+'\zamowienia_n13')
    name='_'.join(list_of_names[0].split('_')[:8])+'_'
    name+=df['Z_DNIA'].max()[:2]
    name+='_'
    name+=df['Z_DNIA'].max()[3:5]
    if df['Z_DNIA'].max()[:2]+'_'+df['Z_DNIA'].max()[3:5] in list_of_names[-1] and len(list_of_names[-1])>51:
        number=int(list_of_names[-1].split('.')[0].split('_')[-1])
        name+='_'
        name+=str(number+1)
    name+='.csv'
    df.to_csv(r'C:\Users\kgrzebien\Desktop\python_tasks\kupony'+'\zamowienia_n13\\'+name,
             encoding='Cp1250',
            sep="|")
    return

def func_save_ewid_rcp(df):
    """FUNKCJA
        -zapisuje nowo pobrane dane z ewid_rcp
    """
    list_of_names=func_read_list_of_names(r'C:\Users\kgrzebien\Desktop\python_tasks\RCP_tables'+'\ewid_rcp')
    name='_'.join(list_of_names[0].split('_')[:2])+'_'
    name+=df_ewid_rcp['DATA'].max()[:2]
    name+='_'
    name+=df_ewid_rcp['DATA'].max()[3:5]
    if df_ewid_rcp['DATA'].max()[:2]+'_'+df_ewid_rcp['DATA'].max()[3:5] in list_of_names[-1] and len(list_of_names[-1])>18:
        number=int(list_of_names[-1].split('.')[0].split('_')[-1])
        name+='_'
        name+=str(number+1)
    else:
        name+='_'
        name+=str(2)
    name+='.csv'
    df.to_csv(r'C:\Users\kgrzebien\Desktop\python_tasks\RCP_tables\ewid_rcp\\'+name,
             encoding='Cp1250',
            sep="|")
    return

In [7]:
def func_back_to_date_as_string(df, date_col):
    """FUNKCJA
        -python automatycznie parsuje date do daty, a potrzebujemy wrocic do stringa w formacie jak w bazie
    """
    df['Z_DNIA2']=df[date_col].astype(str)
    for i in df.index:
        if df.loc[i, date_col].month>=10:
            if df.loc[i, date_col].day>=10:
                df.loc[i,'Z_DNIA2']=str(df.loc[i, date_col].year)[-2:]+'/'+\
                                    str(df.loc[i, date_col].month)+'/'+\
                                    str(df.loc[i, date_col].day)
            else:
                df.loc[i,'Z_DNIA2']=str(df.loc[i, date_col].year)[-2:]+'/'+\
                                    str(df.loc[i, date_col].month)+'/'+\
                                    '0'+str(df.loc[i, date_col].day)
        else:
            if df.loc[i, date_col].day>=10:
                df.loc[i,'Z_DNIA2']=str(df.loc[i, date_col].year)[-2:]+'/'+\
                                    '0'+str(df.loc[i, date_col].month)+'/'+\
                                    str(df.loc[i, date_col].day)
            else:
                df.loc[i,'Z_DNIA2']=str(df.loc[i, date_col].year)[-2:]+'/'+\
                                    '0'+str(df.loc[i, date_col].month)+'/'+\
                                    '0'+str(df.loc[i, date_col].day)
    df=df.drop( date_col, axis=1)
    df=df.rename(columns={'Z_DNIA2': date_col})
    return df

In [8]:
def func_read_data_n13(query, date_col):
    """FUNKCJA
        -łączy się z bazą
        -wybiera odpowiednie zapytanie
        -wywołuje zapis nowo pobranych danych
    """
    import cx_Oracle
    try:
        cx_Oracle.init_oracle_client(lib_dir=r"C:\oracle_python\instantclient_19_9")
    except:
        pass
    try:
        host = '192.168.0.206'
        port = 1521
        SID = 'orcl'

        dsn = cx_Oracle.makedsn(host, port, SID)

        connection = cx_Oracle.connect(user="EF_KG",
                                      password='HgmD5U$CcC',
                                      dsn=dsn,
                                       encoding="UTF-8")

        # show the version of the Oracle Database
        print(connection.version)
        print('pobieram')
        if 'n13' in query:
            df = pd.read_sql(query, con=connection)
        elif 'EWID_RCP' in query:
            df = pd.read_sql(query, con=connection)
        else: pass
        
        df=func_back_to_date_as_string(df, date_col)
        print('skonczylem pobierac')
    except cx_Oracle.Error as error:
        print(error)
    finally:
        # release the connection
        if connection:
            connection.close()
    
    try:
        if 'n13' in query:
            func_save_last_n13(df)
            print('zapisano')
        elif 'EWID_RCP' in query:
            func_save_ewid_rcp(df)
            print('zapisano')
        else: pass
    except: print('error zapis')
    
    return df

In [9]:
def func_read_list_of_names(dir_path):
    """FUNKCJA
        -zczytuje nazwy plikow pod ścieżką
    """
    import os

    res = []
    #dir_path=r'C:\Users\kgrzebien\Desktop\python_tasks\kupony'+'\zamowienia_n13'
    
    for path in os.listdir(dir_path):
        if os.path.isfile(os.path.join(dir_path, path)):
            res.append(path)
    res=np.sort(res)
    return res

In [98]:
"""KOD
        -wczytuje zapisane pliki z n13
        -appenduje wczytane pliki
    """
list_of_names=func_read_list_of_names(r'C:\Users\kgrzebien\Desktop\python_tasks\kupony'+'\zamowienia_n13')
df=func_append_csv(list_of_names,
                   'Z_DNIA',
                  '../zamowienia_n13/')

22/08/04
22/08/17
22/08/19
22/08/22
22/08/23
22/08/24
22/08/25
22/08/26
22/08/29


In [99]:
"""KOD
        -wywołuje pobieranie danych z n13
    """
query_new_n13=func_query('n13',
                           df['Z_DNIA'].max())
df_append=func_read_data_n13(query_new_n13, 'Z_DNIA')

11.2.0.4.0
pobieram
skonczylem pobierac
zapisano


In [100]:
"""KOD
        -appenduje pobrane dane
    """
df=func_append_last_n13(df, df_append, 'Z_DNIA')

In [101]:
"""KOD
        -uruchamia preprocessing
    """
df['ILOSC']=func_str_to_float(df['ILOSC'])
df['WARTOSC_B']=func_str_to_float(df['WARTOSC_B'])
df['WARTOSC_N']=func_str_to_float(df['WARTOSC_N'])
df=func_date(df, '2020-01-01')
df=func_add_name_workers(df, '../../pracownicy.xlsx')
df=func_create_table(df)
df, df_unikatowe_dni_tnaca = func_add_first_and_last_date_workers_and_unique_days(df, 'TNACA1_NAZWISKO')

df_with_time_per_fv=func_time_for_fv(df)
df=df.loc[df['STOL']!='0-0'].reset_index().drop('index', axis=1)

df=func_add_rozmiarsi(df, '../../słowniki/slownikid.csv')

In [102]:
"""KOD
        -prawdopodbne zmiany identyfikatorow
    """
df.loc[df.loc[df['TNACA1']==1840].index, 'TNACA1']=595
df.loc[df.loc[df['TNACA1']==2093].index, 'TNACA1']=537
df.loc[df.loc[df['TNACA1']==318].index, 'TNACA1']=1841
df.loc[df.loc[df['TNACA2']==1840].index, 'TNACA2']=595
df.loc[df.loc[df['TNACA2']==2093].index, 'TNACA2']=537
df.loc[df.loc[df['TNACA2']==318].index, 'TNACA2']=1841

# RCP

## rcp read & download

In [60]:
"""FUNKCJA
        -preprocessing rcp
    """
def func_date_rcp(df):
    
    df=df.loc[df['GODZ_ROZ']!=' '].loc[df['GODZ_ZAK']!=' '].reset_index().drop('index', axis=1)
    
    df['date_start']=df['DATA']+' '+df['GODZ_ROZ']
    df['date_start']=pd.to_datetime(df['date_start'], format='%y/%m/%d %H:%M:%S')
    df['date_start']=df['date_start'].dt.floor('Min')
    df['date_finish']=df['DATA']+' '+df['GODZ_ZAK']
    df['date_finish']=pd.to_datetime(df['date_finish'], format='%y/%m/%d %H:%M:%S')
    df['date_finish']=df['date_finish'].dt.floor('Min')
    df['DATA']=pd.to_datetime(df['DATA'], format='%y/%m/%d')
    
    return df



def func_correct_time(df_ewid_rcp):
    ind=df_ewid_rcp.loc[df_ewid_rcp['GODZ_ROZ']==df_ewid_rcp['GODZ_ZAK']].loc[df_ewid_rcp['ID_ZAD']=='ZP0000003424'].index
    for i in ind:
        new_hour_finish=str(int(df_ewid_rcp.loc[i, 'GODZ_ROZ'][:2])+8)
        df_ewid_rcp.loc[i, 'GODZ_ZAK']=new_hour_finish+':00:00'
    
        
    df_ewid_rcp=df_ewid_rcp.loc[df_ewid_rcp['GODZ_ROZ']<=df_ewid_rcp['GODZ_ZAK']]
    
    ind=df_ewid_rcp.loc[df_ewid_rcp['GODZ_ROZ']==df_ewid_rcp['GODZ_ZAK']].loc[df_ewid_rcp['ID_ZAD']=='ZP0000000996'].index
    for i in ind:
        s=df_ewid_rcp.loc[i, 'GODZ_ROZ']
        df_ewid_rcp.loc[i, 'GODZ_ZAK']=s[:3]+'0'+str(int(s[3:5])+1)+s[5:]
    return df_ewid_rcp

In [103]:
"""KOD
        -wczytuje zapisane pliki z ewid
        -appenduje wczytane pliki
    """
list_of_names=func_read_list_of_names(r'C:\Users\kgrzebien\Desktop\python_tasks\RCP_tables'+'\ewid_rcp')
df_ewid_rcp=func_append_csv(list_of_names,
                   'DATA',
                  '../../RCP_tables/ewid_rcp/')


22/08/16
22/08/22
22/08/23
22/08/24
22/08/25
22/08/27
22/08/29
22/09/01


In [104]:
"""KOD
        -wywołuje pobieranie danych z ewid
    """
#%%time
query_ewid=func_query('rcp',
                           df_ewid_rcp['DATA'].max())
df_ewid_rcp_append=func_read_data_n13(query_ewid, 'DATA')

df_ewid_rcp_append=df_ewid_rcp_append.drop('NEW_DATE', axis=1)

11.2.0.4.0
pobieram
skonczylem pobierac
zapisano


In [105]:
"""KOD
        -appenduje pobrane dane
    """
df_ewid_rcp=df_ewid_rcp[df_ewid_rcp_append.columns]
df_ewid_rcp=func_append_last_n13(df_ewid_rcp, df_ewid_rcp_append, 'DATA')

In [107]:
print(len(df_ewid_rcp[df_ewid_rcp.duplicated()]))
df_ewid_rcp=df_ewid_rcp.drop_duplicates()

1370


In [108]:
"""KOD
        -wczytuje pozostałę dane z rcp
    """
#%%time

def func_read_csv(name):
    df=pd.read_csv(name,
                  encoding='Cp1250',
                  sep="|")
    df=df.loc[df['IS_DELETED']=='N']
    return df


df_grup_rcp=func_read_csv('../../RCP_tables/GRUP_RCP.csv')
df_zad_rcp=func_read_csv('../../RCP_tables/ZAD_RCP.csv')
# df_ewid_rcp=func_append_csv(['../../RCP_tables/EWID_RCP_08_22.csv',
#                             '../../RCP_tables/EWID_RCP_08_22_2.csv'],
#                                'DATA')
df_ewid_rcp=df_ewid_rcp.loc[df_ewid_rcp['GODZ_ROZ']!=' '].loc[df_ewid_rcp['GODZ_ZAK']!=' ']

df_ewid_rcp=func_correct_time(df_ewid_rcp)
df_ewid_rcp=func_date_rcp(df_ewid_rcp)

df_ewid_rcp_total=df_ewid_rcp.copy()
df_ewid_rcp_total=df_ewid_rcp_total.loc[(df_ewid_rcp_total['DATA']>=df['Z_DNIA'].min()) & (df_ewid_rcp_total['DATA']<=df['Z_DNIA'].max())]

df_ewid_rcp=df_ewid_rcp.loc[(df_ewid_rcp['KOD_OPER'].isin(df['TNACA1'].unique()))|(df_ewid_rcp['KOD_OPER'].isin(df['TNACA2'].unique()))]
df_ewid_rcp=df_ewid_rcp.loc[(df_ewid_rcp['DATA']>=df['Z_DNIA'].min()) & (df_ewid_rcp['DATA']<=df['Z_DNIA'].max())]

df_rcp=df_ewid_rcp[['ID_ZAD','DATA','ROZ_DNIA','ZAK_DNIA','GODZ_PL','GODZ_ROZ','GODZ_ZAK','KOD_OPER', 'date_start', 'date_finish']].\
                merge(df_zad_rcp[['ID_ZAD','ZADANIE']].drop_duplicates(),
                      how='left', on='ID_ZAD')
df_rcp_total=df_ewid_rcp_total[['ID_ZAD','DATA','ROZ_DNIA','ZAK_DNIA','GODZ_PL','GODZ_ROZ','GODZ_ZAK','KOD_OPER', 'date_start', 'date_finish']].\
                merge(df_zad_rcp[['ID_ZAD','ZADANIE']].drop_duplicates(),
                      how='left', on='ID_ZAD')

In [109]:
df_operatorzy_do_usuniecia_z_rcp=df_ewid_rcp.loc[df_ewid_rcp['KOD_OPER'].isin(np.sort(df['TNACA1'].unique()))].\
                                            groupby(['KOD_OPER','DATA']).agg(KOD_GR_RCP_unique=('KOD_GR_RCP','unique'),
                                                                            KOD_GR_RCP_nunique=('KOD_GR_RCP','nunique')).reset_index().\
                                            merge(df[['TNACA1','TNACA1_NAZWISKO']].drop_duplicates(), how='left', left_on='KOD_OPER', right_on='TNACA1')#.to_excel(path_to_catalog+'usun_operator_czy_caly_dzien.xlsx')

df_operatorzy_do_usuniecia_z_rcp['KOD_GR_RCP_unique_str']=df_operatorzy_do_usuniecia_z_rcp['KOD_GR_RCP_unique'].apply(lambda x: ','.join(x))
df_operatorzy_do_usuniecia_z_rcp=df_operatorzy_do_usuniecia_z_rcp.loc[df_operatorzy_do_usuniecia_z_rcp['KOD_GR_RCP_nunique']==1].\
                                                                loc[~((df_operatorzy_do_usuniecia_z_rcp['KOD_GR_RCP_unique_str'].str.contains('KUP')) | (df_operatorzy_do_usuniecia_z_rcp['KOD_GR_RCP_unique_str'].str.contains('L4')))]\
                                    [['KOD_OPER','TNACA1_NAZWISKO']].drop_duplicates().sort_values(by='TNACA1_NAZWISKO').reset_index()
df_operatorzy_do_usuniecia_z_rcp=df_operatorzy_do_usuniecia_z_rcp.loc[df_operatorzy_do_usuniecia_z_rcp['TNACA1_NAZWISKO']!='G. ŁUKAŃKO']

In [110]:
df_rcp['zadanie_group']=df_rcp['ZADANIE'].apply(lambda x: 'KUPONY-REALIZACJA ZAMÓWIENIA' if x=='KUPONY-REALIZACJA ZAMÓWIENIA' \
                                                                                       else ('kupony_inne' if x[:6]=='KUPONY'\
                                                                                                            else 'pomoc' if x[-5:]=='POMOC' \
                                                                                                                         else ('nieobecnosc' if x in ['CHOROBOWE', 'NIEOBECNOŚĆ CAŁODNIOWA USPRAWI', 'OPIEKA', 'URLOP'] else 'inne')))
df_rcp_total['zadanie_group']=df_rcp_total['ZADANIE']


## rcp calc

In [111]:
"""FUNKCJA
        -wyliczenie minut poswiecanych na dana czynnosc w ciagu dnia
    """
def func_minutes_in_work_day_rcp(df, tnaca1_numer, min_date):
    step=np.datetime64('2000-01-01 00:02:00')-np.datetime64('2000-01-01 00:01:00')
    
    df_with_minutes=pd.DataFrame(columns=['KOD_OPER', 'DATA', 'ile_minut_czynnosc', 'ile_minut_dzien', 'procent_wykorzystania', 'zadanie','zadanie_group'])
    
    for day in (df.loc[df['KOD_OPER']==tnaca1_numer]['DATA'].unique()):
        if day<=min_date:#np.datetime64('2022-07-01'):
            continue
        else:
            arr_time_day=np.array([], dtype='datetime64')
            arr_time_activity=np.array([], dtype='int')
            arr_zadanie=[]
            arr_zadanie_group=[]
            try:
                for i in (df.loc[df['KOD_OPER']==tnaca1_numer].loc[df['DATA']==day].index):
                    arr_tmp=np.arange(df.loc[i,'date_start'],
                                      df.loc[i,'date_finish'],
                                     step)
                    arr_time_day=np.append(arr_time_day, arr_tmp)
                    arr_time_activity=np.append(arr_time_activity, [len(arr_tmp)])
                    arr_zadanie=np.append(arr_zadanie, [df.loc[i,'ZADANIE']])
                    arr_zadanie_group=np.append(arr_zadanie_group, [df.loc[i,'zadanie_group']])

                if len(arr_time_day)==0:
                    display(day)
                ile_minut_maksymalnie=(arr_time_day.max()-arr_time_day.min())/np.timedelta64(1,'m')+1
                for time_activity, zadanie, zadanie_group in zip(arr_time_activity, arr_zadanie, arr_zadanie_group ):
                    df_with_minutes=df_with_minutes.append({'KOD_OPER':tnaca1_numer,
                                                            'DATA':day,
                                                            'ile_minut_czynnosc':time_activity,
                                                            'ile_minut_dzien':ile_minut_maksymalnie,
                                                           'procent_wykorzystania':time_activity/ile_minut_maksymalnie*100, 
                                                            'zadanie':zadanie,
                                                            'zadanie_group':zadanie_group},
                                                           ignore_index=True)
            except:
                pass

    
    return df_with_minutes

In [112]:
"""FUNKCJA
        -wywołuje wyliczenia minutowe
    """
def func_run_calc_minutes_rcp(min_date):
#tmp_tnaca_1688 = func_minutes_in_work_day_rcp(df_rcp, 1688)

    df_with_minutes_rcp=pd.DataFrame()
    #flaga=0
    for tnaca1 in tqdm(np.unique(np.append(df['TNACA1'].unique(), df['TNACA2'].unique()))):
        try:
            if tnaca1 == 0:
                continue

            df_with_minutes_tmp=func_minutes_in_work_day_rcp(df_rcp,
                                                                 tnaca1,
                                                            min_date)
            df_with_minutes_rcp=df_with_minutes_rcp.append(df_with_minutes_tmp,
                                                              ignore_index=True)
        except:
            print(tnaca1)
            continue
        # if flaga==3:
        #     break
        # else:
        #     flaga+=1
    return df_with_minutes_rcp

In [113]:
"""KOD
        -wczytuje stare dane minutowe rcp
        -wywołuje wyliczenia minutowe na nowych danych
        -appenduje wyliczenia minutowe
    """
df_with_minutes_rcp_old=pd.read_excel(path_to_catalog+'df_with_minutes_rcp_2.xlsx').drop('Unnamed: 0', axis=1)
df_with_minutes_rcp_old['DATA']=pd.to_datetime(df_with_minutes_rcp_old['DATA'], format='%y-%m-%d')

df_with_minutes_rcp=func_run_calc_minutes_rcp(df_with_minutes_rcp_old['DATA'].max())

df_with_minutes_rcp=df_with_minutes_rcp.append(df_with_minutes_rcp_old, ignore_index=True)

100%|████████████████████████████████████████████████████████████████████████████████| 104/104 [01:57<00:00,  1.13s/it]


In [114]:
"""KOD
        -zapisuje appendowane wyliczenia minutowe
        -bedzie trzeba to robic co jakis czas
        -nie napisalem automatu
    """
# df_with_minutes_rcp.to_excel(path_to_catalog+'df_with_minutes_rcp_3.xlsx')

'KOD\n        -zapisuje appendowane wyliczenia minutowe\n        -bedzie trzeba to robic co jakis czas\n        -nie napisalem automatu\n    '

In [115]:
"""KOD
        -preprocessing danych minutowych
    """
def func_year_month_from_rcp(df):
    df['year']=df['DATA'].apply(lambda x: x.year)
    df['month']=df['DATA'].apply(lambda x: x.month)
    df['year_month']=df['year'].astype(str)+'-'+df['month'].astype(str)+'-'+'01'
    df['year_month']=df['year_month'].apply(lambda x: x[:5]+'0'+x[5:] if len(x)==9 else x)
    df['year_month']=pd.to_datetime(df['year_month'], format='%Y-%m-%d')
    df['week']=df['DATA'].apply(lambda x: x.isocalendar()[1])
    df['year_week']=df['year'].astype(str)+'--'+df['week'].astype(str)
    df['year_week']=df['year_week'].apply(lambda x: x[:6]+'0'+x[6:] if len(x)==7 else x)
    return df

df_with_minutes_rcp=func_year_month_from_rcp(df_with_minutes_rcp)

#df_with_minutes_rcp_probniki=func_year_month_from_rcp(df_with_minutes_rcp_probniki)

In [116]:
"""KOD
        -pogrupowanie zadan w rcp
    """
df_with_minutes_rcp['zadania_ogolnie_5_grupy']=df_with_minutes_rcp['zadanie'].apply(lambda x: '1 - KUPONY-REALIZACJA ZAMÓWIENIA' if x=='KUPONY-REALIZACJA ZAMÓWIENIA' \
                                                                                       else ('1 - kupony_inne' if (x[:6]=='KUPONY' or x=='PRZERWA')\
                                                                                                            else 'inne' if x[-5:]=='POMOC' \
                                                                                                                         else ('nieobecnosc' if x in ['CHOROBOWE', 'NIEOBECNOŚĆ CAŁODNIOWA USPRAWI', 'OPIEKA', 'URLOP',
                                                                                                                                                     'URLOP OKOLICZNOŚCIOWY - urlop','URLOP OKOLICZNOŚCIOWY',
                                                                                                                                                      'Wyjście prywatne','IZOLACJA DOMOWA',
                                                                                                                                                      'L4-OPIEKA','OPIEKA - KOR-8LAT','ZASIŁEK','KWARANTANNA','WOLNE ZA ŚWIĘTO',
                                                                                                                                                      'N-PRACA ZDALNA','NN'] \
                                                                                                                                               else ('targi' if x in ['TARGI'] else 'inne'))))
df_with_minutes_rcp['zadania_ogolnie_3_grupy']=df_with_minutes_rcp['zadanie'].apply(lambda x: '1-kupony' if (x[:6]=='KUPONY' or x=='PRZERWA') \
                                                                                                        else '2-inne' if x[-5:]=='POMOC' \
                                                                                                                    else ('3-nieobecnosc' if x in ['CHOROBOWE', 'NIEOBECNOŚĆ CAŁODNIOWA USPRAWI', 'OPIEKA', 'URLOP',
                                                                                                                                                     'URLOP OKOLICZNOŚCIOWY - urlop','URLOP OKOLICZNOŚCIOWY',
                                                                                                                                                   'Wyjście prywatne','IZOLACJA DOMOWA',
                                                                                                                                                      'L4-OPIEKA','OPIEKA - KOR-8LAT','ZASIŁEK','KWARANTANNA','WOLNE ZA ŚWIĘTO',
                                                                                                                                                      'N-PRACA ZDALNA','NN'] \
                                                                                                                                            else '2-inne'))

df_with_minutes_rcp['zadania_ogolnie_4_grupy']=df_with_minutes_rcp['zadanie'].apply(lambda x: '1-kupony' if (x[:6]=='KUPONY' or x=='PRZERWA') \
                                                                                                        else '2-pomoc' if x[-5:]=='POMOC' \
                                                                                                                    else ('4-nieobecnosc' if x in ['CHOROBOWE', 'NIEOBECNOŚĆ CAŁODNIOWA USPRAWI', 'OPIEKA', 'URLOP',
                                                                                                                                                     'URLOP OKOLICZNOŚCIOWY - urlop','URLOP OKOLICZNOŚCIOWY',
                                                                                                                                                   'Wyjście prywatne','IZOLACJA DOMOWA',
                                                                                                                                                      'L4-OPIEKA','OPIEKA - KOR-8LAT','ZASIŁEK','KWARANTANNA','WOLNE ZA ŚWIĘTO',
                                                                                                                                                      'N-PRACA ZDALNA','NN'] \
                                                                                                                                            else '3-inne'))

# df_with_minutes_rcp['zadania_ogolnie_przerwy_inne']=df_with_minutes_rcp['zadanie'].apply(lambda x: 'PRZERWA' if x=='PRZERWA' \
#                                                                                        else 'praca')

In [117]:
"""FUNKCJA
        -pogrupowanie czasów w sposób względny
    """
def func_minutes_rcp_group(df_with_minutes_rcp,
                          arr_group,
                          wzgledny_czy_bezwzgledny):
    
    if wzgledny_czy_bezwzgledny=='wzgledny':
        if 'KOD_OPER' in arr_group:
            _tmp=df_with_minutes_rcp.groupby([arr_group[0]]+['DATA','KOD_OPER']).agg({'ile_minut_dzien':'first'}).reset_index().groupby([arr_group[0]]+['KOD_OPER']).agg({'ile_minut_dzien':'sum'}).reset_index()
            _tmp_nieobecnosc=df_with_minutes_rcp.loc[df_with_minutes_rcp[arr_group[1]].str.contains('nieobecnosc')].groupby([arr_group[0]]+['KOD_OPER']).agg(czas_nieobecnosc=('ile_minut_czynnosc','sum')).reset_index()#groupby([arr_group[0]]+['DATA','KOD_OPER']).agg({'ile_minut_dzien':'first'}).reset_index().\
                                                                                                                    
            _tmp=_tmp.merge(_tmp_nieobecnosc, how='left', on=[arr_group[0]]+['KOD_OPER'])
            _tmp['czas_nieobecnosc']=_tmp['czas_nieobecnosc'].fillna(0)
            _tmp['ile_minut_dzien']=_tmp['ile_minut_dzien']-_tmp['czas_nieobecnosc']
            _tmp=_tmp[[arr_group[0]]+['KOD_OPER']+['ile_minut_dzien']]
        elif arr_group[0]=='DATA':
            _tmp=df_with_minutes_rcp.groupby([arr_group[0]]+['KOD_OPER']).agg({'ile_minut_dzien':'first'}).reset_index().groupby(arr_group[0]).agg({'ile_minut_dzien':'sum'}).reset_index()
        else:
            _tmp=df_with_minutes_rcp.groupby([arr_group[0]]+['DATA','KOD_OPER']).agg({'ile_minut_dzien':'first'}).reset_index().groupby(arr_group[0]).agg({'ile_minut_dzien':'sum'}).reset_index()
        
        df_with_minutes_rcp_group=df_with_minutes_rcp.groupby(arr_group).agg({'ile_minut_czynnosc':'sum'}).reset_index()
        if 'KOD_OPER' in arr_group:
            df_with_minutes_rcp_group=df_with_minutes_rcp_group.merge(_tmp, how='left', on=[arr_group[0]]+['KOD_OPER'])
        else:
            df_with_minutes_rcp_group=df_with_minutes_rcp_group.merge(_tmp, how='left', on=arr_group[0])
            
        # to trzeba wywalic i naprawic o co chodzi bo robi czas dnia 0 wtf
        try:
            df_with_minutes_rcp_group=df_with_minutes_rcp_group.loc[df_with_minutes_rcp_group['zadania_ogolnie_4_grupy']!='4-nieobecnosc']
        except:
            df_with_minutes_rcp_group=df_with_minutes_rcp_group.loc[df_with_minutes_rcp_group['zadania_ogolnie_5_grupy']!='nieobecnosc']
        df_with_minutes_rcp_group['procent_wykorzystania']=df_with_minutes_rcp_group['ile_minut_czynnosc']/df_with_minutes_rcp_group['ile_minut_dzien']*100
    else:
        df_with_minutes_rcp_group=df_with_minutes_rcp.groupby(arr_group).agg({'procent_wykorzystania':'sum'}).reset_index()
        
        
    return df_with_minutes_rcp_group
        

In [118]:
df_with_minutes_rcp_group=func_minutes_rcp_group(df_with_minutes_rcp.loc[~df_with_minutes_rcp['KOD_OPER'].isin(df_operatorzy_do_usuniecia_z_rcp['KOD_OPER'].unique())],
                                                  ['year_month','zadania_ogolnie_4_grupy'],
                                                  'wzgledny')

# calc table

## calc func

In [119]:
def func_tnaca_nazwisko(df):
    #display(df)
    df1=df[['TNACA1','TNACA1_NAZWISKO']]
    df2=df[['TNACA2','TNACA2_NAZWISKO']]
    df1=df1.rename(columns={'TNACA1':'tnaca','TNACA1_NAZWISKO':'TNACA_NAZWISKO'})
    df2=df2.rename(columns={'TNACA2':'tnaca','TNACA2_NAZWISKO':'TNACA_NAZWISKO'})
    df=df1.append(df2, ignore_index=True)
    return df.drop_duplicates()
    
def func_rcp_proc_time(year1, year2, month, df_with_name, mc_or_total):
    if mc_or_total=='mc':
        tmp=func_minutes_rcp_group(df_with_minutes_rcp.loc[~df_with_minutes_rcp['KOD_OPER'].isin(df_operatorzy_do_usuniecia_z_rcp['KOD_OPER'].unique())],
                                                          ['year_month','zadania_ogolnie_4_grupy','KOD_OPER'],
                                                          'wzgledny')
    else:
        tmp=func_minutes_rcp_group(df_with_minutes_rcp.loc[df_with_minutes_rcp['month'].isin(month)].loc[~df_with_minutes_rcp['KOD_OPER'].isin(df_operatorzy_do_usuniecia_z_rcp['KOD_OPER'].unique())],
                                                          ['year','zadania_ogolnie_4_grupy','KOD_OPER'],
                                                          'wzgledny')
    tmp=tmp.loc[tmp['zadania_ogolnie_4_grupy']=='1-kupony']
    
    
    
    if mc_or_total=='mc':
        tmp=tmp[['year_month','KOD_OPER','procent_wykorzystania']]
        tmp['year']=tmp['year_month'].apply(lambda x: x.year)
        tmp['month']=tmp['year_month'].apply(lambda x: x.month)
        tmp1=tmp.loc[tmp['year']==year1].loc[tmp['month'].isin(month)]
        tmp2=tmp.loc[tmp['year']==year2].loc[tmp['month'].isin(month)]
        tmp1=tmp1.drop(['year_month','year'], axis=1)
        tmp2=tmp2.drop(['year_month','year'], axis=1)
    else:
        tmp=tmp[['year','KOD_OPER','procent_wykorzystania']]
        tmp1=tmp.loc[tmp['year']==year1]
        tmp2=tmp.loc[tmp['year']==year2]
        tmp1=tmp1.drop(['year'], axis=1)
        tmp2=tmp2.drop(['year'], axis=1)
        
    tmp1=tmp1.rename(columns={'procent_wykorzystania':'procent_wykorzystania_czasu_pracy_na_kupony_'+str(year1)})
    tmp2=tmp2.rename(columns={'procent_wykorzystania':'procent_wykorzystania_czasu_pracy_na_kupony_'+str(year2)})
    if mc_or_total=='mc':
        tmp=tmp1.merge(tmp2, how='outer', on=['KOD_OPER','month'])
    else:
        tmp=tmp1.merge(tmp2, how='outer', on=['KOD_OPER'])
        
    tmp=tmp.merge(df_with_name, how='left', left_on='KOD_OPER', right_on='tnaca')
    return tmp


def func_rcp_time_real_zamowienia(year1, year2, month, df_with_name, mc_or_total):
    if mc_or_total=='mc':
        tmp=func_minutes_rcp_group(df_with_minutes_rcp.loc[~df_with_minutes_rcp['KOD_OPER'].isin(df_operatorzy_do_usuniecia_z_rcp['KOD_OPER'].unique())],
                                                          ['year_month','zadania_ogolnie_5_grupy','KOD_OPER'],
                                                          'wzgledny')
    else:
        tmp=func_minutes_rcp_group(df_with_minutes_rcp.loc[df_with_minutes_rcp['month'].isin(month)].loc[~df_with_minutes_rcp['KOD_OPER'].isin(df_operatorzy_do_usuniecia_z_rcp['KOD_OPER'].unique())],
                                                          ['year','zadania_ogolnie_5_grupy','KOD_OPER'],
                                                          'wzgledny')
    tmp=tmp.loc[tmp['zadania_ogolnie_5_grupy']=='1 - KUPONY-REALIZACJA ZAMÓWIENIA']
    
    
    
    if mc_or_total=='mc':
        tmp=tmp[['year_month','KOD_OPER','ile_minut_czynnosc']]
        tmp['year']=tmp['year_month'].apply(lambda x: x.year)
        tmp['month']=tmp['year_month'].apply(lambda x: x.month)
        tmp1=tmp.loc[tmp['year']==year1].loc[tmp['month'].isin(month)]
        tmp2=tmp.loc[tmp['year']==year2].loc[tmp['month'].isin(month)]
        tmp1=tmp1.drop(['year_month','year'], axis=1)
        tmp2=tmp2.drop(['year_month','year'], axis=1)
    else:
        tmp=tmp[['year','KOD_OPER','ile_minut_czynnosc']]
        tmp1=tmp.loc[tmp['year']==year1]
        tmp2=tmp.loc[tmp['year']==year2]
        tmp1=tmp1.drop(['year'], axis=1)
        tmp2=tmp2.drop(['year'], axis=1)
        
    tmp1=tmp1.rename(columns={'ile_minut_czynnosc':'ile_minut_czynnosc_na_realizacje_kupony_'+str(year1)})
    tmp2=tmp2.rename(columns={'ile_minut_czynnosc':'ile_minut_czynnosc_na_realizacje_kupony_'+str(year2)})
    if mc_or_total=='mc':
        tmp=tmp1.merge(tmp2, how='outer', on=['KOD_OPER','month'])
    else:
        tmp=tmp1.merge(tmp2, how='outer', on=['KOD_OPER'])
        
    tmp=tmp.merge(df_with_name, how='left', left_on='KOD_OPER', right_on='tnaca')
    return tmp


def func_rcp_time_cale_kupony(year1, year2, month, df_with_name, mc_or_total):
    if mc_or_total=='mc':
        tmp=func_minutes_rcp_group(df_with_minutes_rcp.loc[~df_with_minutes_rcp['KOD_OPER'].isin(df_operatorzy_do_usuniecia_z_rcp['KOD_OPER'].unique())],
                                                          ['year_month','zadania_ogolnie_4_grupy','KOD_OPER'],
                                                          'wzgledny')
    else:
        tmp=func_minutes_rcp_group(df_with_minutes_rcp.loc[df_with_minutes_rcp['month'].isin(month)].loc[~df_with_minutes_rcp['KOD_OPER'].isin(df_operatorzy_do_usuniecia_z_rcp['KOD_OPER'].unique())],
                                                          ['year','zadania_ogolnie_4_grupy','KOD_OPER'],
                                                          'wzgledny')
    tmp=tmp.loc[tmp['zadania_ogolnie_4_grupy']=='1-kupony']
    
    
    
    if mc_or_total=='mc':
        tmp=tmp[['year_month','KOD_OPER','ile_minut_czynnosc']]
        tmp['year']=tmp['year_month'].apply(lambda x: x.year)
        tmp['month']=tmp['year_month'].apply(lambda x: x.month)
        tmp1=tmp.loc[tmp['year']==year1].loc[tmp['month'].isin(month)]
        tmp2=tmp.loc[tmp['year']==year2].loc[tmp['month'].isin(month)]
        tmp1=tmp1.drop(['year_month','year'], axis=1)
        tmp2=tmp2.drop(['year_month','year'], axis=1)
    else:
        tmp=tmp[['year','KOD_OPER','ile_minut_czynnosc']]
        tmp1=tmp.loc[tmp['year']==year1]
        tmp2=tmp.loc[tmp['year']==year2]
        tmp1=tmp1.drop(['year'], axis=1)
        tmp2=tmp2.drop(['year'], axis=1)
        
    tmp1=tmp1.rename(columns={'ile_minut_czynnosc':'ile_minut_czynnosc_na_cale_kupony_'+str(year1)})
    tmp2=tmp2.rename(columns={'ile_minut_czynnosc':'ile_minut_czynnosc_na_cale_kupony_'+str(year2)})
    if mc_or_total=='mc':
        tmp=tmp1.merge(tmp2, how='outer', on=['KOD_OPER','month'])
    else:
        tmp=tmp1.merge(tmp2, how='outer', on=['KOD_OPER'])
        
    tmp=tmp.merge(df_with_name, how='left', left_on='KOD_OPER', right_on='tnaca')
    return tmp

In [136]:
def func_create_pivot_table_agata(df, year1, year2, month, tnaca_czy_stol, czy_test):
    
    df_rcp_proc_time=func_rcp_proc_time(year1, year2, month, func_tnaca_nazwisko(df), 'mc')
    df_rcp_proc_time_total=func_rcp_proc_time(year1, year2, month, func_tnaca_nazwisko(df), 'total')
    df_rcp_time_cale_kupony_godziny=func_rcp_time_cale_kupony(year1, year2, month, func_tnaca_nazwisko(df), 'mc')
    df_rcp_time_cale_kupony_godziny_total=func_rcp_time_cale_kupony(year1, year2, month, func_tnaca_nazwisko(df), 'total')
    df_rcp_time_real_zamowienia_godziny=func_rcp_time_real_zamowienia(year1, year2, month, func_tnaca_nazwisko(df), 'mc')
    df_rcp_time_real_zamowienia_godziny_total=func_rcp_time_real_zamowienia(year1, year2, month, func_tnaca_nazwisko(df), 'total')
    
    df_table_pivot=pd.DataFrame(columns=['TNACA_NAZWISKO','month',
                                         'FAKTURA_'+str(year1),'FAKTURA_'+str(year2), 'FAKTURA_DYNAMIKA',
                                         'ILOSC_'+str(year1),'ILOSC_'+str(year2), 'ILOSC_DYNAMIKA',
                                         'ROZMIARSI_'+str(year1), 'ROZMIARSI_'+str(year2), 'ROZMIARSI_DYNAMIKA',
                                         'WARTOSC_B_'+str(year1),'WARTOSC_B_'+str(year2), 'WARTOSC_B_DYNAMIKA',
                                         'KOD_TEMA_'+str(year1), 'KOD_TEMA_'+str(year2), 'KOD_TEMA_DYNAMIKA',
                                        'PROCENT_CZASU_PRACY_NA_KUPONACH_'+str(year1), 'PROCENT_CZASU_PRACY_NA_KUPONACH_'+str(year2), 'PROCENT_CZASU_PRACY_NA_KUPONACH_DYNAMIKA',
                                         'LICZBA_GODZIN_NA_CALYCH_KUPONACH_'+str(year1), 'LICZBA_GODZIN_NA_CALYCH_KUPONACH_'+str(year2), 'LICZBA_GODZIN_NA_CALYCH_KUPONACH_DYNAMIKA',
                                         'LICZBA_GODZIN_NA_REAL_ZAMOWIEN_'+str(year1), 'LICZBA_GODZIN_NA_REAL_ZAMOWIEN_'+str(year2), 'LICZBA_GODZIN_NA_REAL_ZAMOWIEN_DYNAMIKA',
                                        'LICZBA_UNIKATOWYCH_DNI_'+str(year1),'LICZBA_UNIKATOWYCH_DNI_'+str(year2), 'LICZBA_UNIKATOWYCH_DNI_DYNAMIKA'])#,
                                        #'LICZBA_UNIKATOWYCH_MINUT_'+str(year1),'LICZBA_UNIKATOWYCH_MINUT_'+str(year2), 'LICZBA_UNIKATOWYCH_MINUT_DYNAMIKA',
                                        #'SREDNIA_LICZBA_UNIKATOWYCH_MINUT_'+str(year1),'SREDNIA_LICZBA_UNIKATOWYCH_MINUT_'+str(year2), 'SREDNIA_LICZBA_UNIKATOWYCH_MINUT_DYNAMIKA'])
    
    if tnaca_czy_stol=='tnaca':
        arr_tnaca_unique=np.append(df['TNACA1_NAZWISKO'].unique(),df['TNACA2_NAZWISKO'].unique())
        arr_tnaca_unique=np.unique(arr_tnaca_unique)
    elif tnaca_czy_stol=='stol':
        arr_tnaca_unique=df.loc[df['year']==year2]['STOL_NAZWISKO'].unique()
        
    else:
        print('BŁĄD parametru: "tnaca_czy_stol"')
        return
    
    for tnaca in tqdm(arr_tnaca_unique):
        if czy_test=="TEST":
            tnaca='S. KAPAŁA'
            tnaca='K. RYPIEŃ'
        for mc in month:
            arr_f=func_get_values_from_global_df(df, tnaca, year1, year2, mc, 'FAKTURA', tnaca_czy_stol)
            arr_i=func_get_values_from_global_df(df, tnaca, year1, year2, mc, 'ILOSC', tnaca_czy_stol)
            arr_w=func_get_values_from_global_df(df, tnaca, year1, year2, mc, 'WARTOSC_B', tnaca_czy_stol)
            arr_k=func_get_values_from_global_df(df, tnaca, year1, year2, mc, 'KOD_TEMA', tnaca_czy_stol)
            arr_r=func_get_values_from_global_df(df, tnaca, year1, year2, mc, 'ROZMIARSI', tnaca_czy_stol)
            arr_day=func_time_unique_day(df, tnaca, year1, year2, mc, tnaca_czy_stol)
            #arr_minutes=func_time_unique_minutes(df, tnaca, year1, year2, mc, tnaca_czy_stol)
            # try:
            #     val_1_srednia_liczba_unikatowych_minut=round(arr_minutes[0]/arr_day[0])
            # except:
            #     val_1_srednia_liczba_unikatowych_minut=0
            # try:
            #     val_2_srednia_liczba_unikatowych_minut=round(arr_minutes[1]/arr_day[1])
            # except:
            #     val_2_srednia_liczba_unikatowych_minut=0
            # try:
            #     val_srednia_liczba_unikatowych_minut_dyn=round((val_2_srednia_liczba_unikatowych_minut-val_1_srednia_liczba_unikatowych_minut)/val_1_srednia_liczba_unikatowych_minut, 2)
            # except:
            #     val_srednia_liczba_unikatowych_minut_dyn=0
            ##
            #procent czasu pracy na kuponach do innych czynnosci
            ##
            try:
                val_1_proc_czasu_pracu=round(df_rcp_proc_time.loc[df_rcp_proc_time['month']==mc].loc[df_rcp_proc_time['TNACA_NAZWISKO']==tnaca]['procent_wykorzystania_czasu_pracy_na_kupony_'+str(year1)].values[0])
            except:
                val_1_proc_czasu_pracu=0
            try:
                val_2_proc_czasu_pracu=round(df_rcp_proc_time.loc[df_rcp_proc_time['month']==mc].loc[df_rcp_proc_time['TNACA_NAZWISKO']==tnaca]['procent_wykorzystania_czasu_pracy_na_kupony_'+str(year2)].values[0])
            except:
                val_2_proc_czasu_pracu=0
            try:
                val_proc_czasu_pracu_dyn=round((val_2_proc_czasu_pracu-val_1_proc_czasu_pracu)/val_1_proc_czasu_pracu, 2)
            except:
                val_proc_czasu_pracu_dyn=0
            ##
            #godziny pracy na calych kuponach 
            ##
            try:
                val_1_godziny_cale_kupony=round(df_rcp_time_cale_kupony_godziny.loc[df_rcp_time_cale_kupony_godziny['month']==mc].loc[df_rcp_time_cale_kupony_godziny['TNACA_NAZWISKO']==tnaca]['ile_minut_czynnosc_na_cale_kupony_'+str(year1)].values[0])
                val_1_godziny_cale_kupony=round(val_1_godziny_cale_kupony/60 , 2)
            except:
                val_1_godziny_cale_kupony=0
            try:
                val_2_godziny_cale_kupony=round(df_rcp_time_cale_kupony_godziny.loc[df_rcp_time_cale_kupony_godziny['month']==mc].loc[df_rcp_time_cale_kupony_godziny['TNACA_NAZWISKO']==tnaca]['ile_minut_czynnosc_na_cale_kupony_'+str(year2)].values[0])
                val_2_godziny_cale_kupony=round(val_2_godziny_cale_kupony/60 , 2)
            except:
                val_2_godziny_cale_kupony=0
            try:
                val_godziny_cale_kupony_dyn=round((val_2_godziny_cale_kupony-val_1_godziny_cale_kupony)/val_1_godziny_cale_kupony, 2)
            except:
                val_godziny_cale_kupony_dyn=0
            ##
            #godziny pracy na realizacji zamowien 
            ##
            try:
                val_1_godziny_real_zamowien=round(df_rcp_time_real_zamowienia_godziny.loc[df_rcp_time_real_zamowienia_godziny['month']==mc].loc[df_rcp_time_real_zamowienia_godziny['TNACA_NAZWISKO']==tnaca]['ile_minut_czynnosc_na_realizacje_kupony_'+str(year1)].values[0])
                val_1_godziny_real_zamowien=round(val_1_godziny_real_zamowien/60 , 2)
            except:
                val_1_godziny_real_zamowien=0
            try:
                val_2_godziny_real_zamowien=round(df_rcp_time_real_zamowienia_godziny.loc[df_rcp_time_real_zamowienia_godziny['month']==mc].loc[df_rcp_time_real_zamowienia_godziny['TNACA_NAZWISKO']==tnaca]['ile_minut_czynnosc_na_realizacje_kupony_'+str(year2)].values[0])
                val_2_godziny_real_zamowien=round(val_2_godziny_real_zamowien/60 , 2)
            except:
                val_2_godziny_real_zamowien=0
            try:
                val_godziny_real_zamowien_dyn=round((val_2_godziny_real_zamowien-val_1_godziny_real_zamowien)/val_1_godziny_real_zamowien, 2)
            except:
                val_godziny_real_zamowien_dyn=0
                
            df_table_pivot=df_table_pivot.append({'TNACA_NAZWISKO':tnaca,
                                                   'month':mc,
                                                    'FAKTURA_'+str(year1):arr_f[0],
                                                    'FAKTURA_'+str(year2):arr_f[1], 
                                                    'FAKTURA_DYNAMIKA':arr_f[2],
                                                    'ILOSC_'+str(year1):arr_i[0],
                                                    'ILOSC_'+str(year2):arr_i[1], 
                                                    'ILOSC_DYNAMIKA':arr_i[2],
                                                    'ROZMIARSI_'+str(year1):round(arr_r[0]/100),
                                                    'ROZMIARSI_'+str(year2):round(arr_r[1]/100), 
                                                    'ROZMIARSI_DYNAMIKA':arr_r[2],
                                                    'WARTOSC_B_'+str(year1):arr_w[0],
                                                    'WARTOSC_B_'+str(year2):arr_w[1], 
                                                    'WARTOSC_B_DYNAMIKA':arr_w[2],
                                                    'KOD_TEMA_'+str(year1):arr_k[0],
                                                    'KOD_TEMA_'+str(year2):arr_k[1], 
                                                    'KOD_TEMA_DYNAMIKA':arr_k[2],
                                                     'PROCENT_CZASU_PRACY_NA_KUPONACH_'+str(year1):val_1_proc_czasu_pracu,
                                                      'PROCENT_CZASU_PRACY_NA_KUPONACH_'+str(year2):val_2_proc_czasu_pracu,
                                                  'PROCENT_CZASU_PRACY_NA_KUPONACH_DYNAMIKA':val_proc_czasu_pracu_dyn,
                                                     'LICZBA_GODZIN_NA_CALYCH_KUPONACH_'+str(year1):val_1_godziny_cale_kupony,
                                                      'LICZBA_GODZIN_NA_CALYCH_KUPONACH_'+str(year2):val_2_godziny_cale_kupony,
                                                  'LICZBA_GODZIN_NA_CALYCH_KUPONACH_DYNAMIKA':val_godziny_cale_kupony_dyn,
                                                     'LICZBA_GODZIN_NA_REAL_ZAMOWIEN_'+str(year1):val_1_godziny_real_zamowien,
                                                      'LICZBA_GODZIN_NA_REAL_ZAMOWIEN_'+str(year2):val_2_godziny_real_zamowien,
                                                  'LICZBA_GODZIN_NA_REAL_ZAMOWIEN_DYNAMIKA':val_godziny_real_zamowien_dyn,  
                                                  'LICZBA_UNIKATOWYCH_DNI_'+str(year1):arr_day[0],
                                                    'LICZBA_UNIKATOWYCH_DNI_'+str(year2):arr_day[1], 
                                                    'LICZBA_UNIKATOWYCH_DNI_DYNAMIKA':arr_day[2]},
                                                 #    'LICZBA_UNIKATOWYCH_MINUT_'+str(year1):arr_minutes[0],
                                                 #    'LICZBA_UNIKATOWYCH_MINUT_'+str(year2):arr_minutes[1], 
                                                 #    'LICZBA_UNIKATOWYCH_MINUT_DYNAMIKA':arr_minutes[2],
                                                 # 'SREDNIA_LICZBA_UNIKATOWYCH_MINUT_'+str(year1):val_1_srednia_liczba_unikatowych_minut,
                                                 #  'SREDNIA_LICZBA_UNIKATOWYCH_MINUT_'+str(year2):val_2_srednia_liczba_unikatowych_minut,
                                                 #  'SREDNIA_LICZBA_UNIKATOWYCH_MINUT_DYNAMIKA':val_srednia_liczba_unikatowych_minut_dyn},
                                                  ignore_index=True)
            
        df_table_pivot=df_table_pivot.fillna(0)
        df_table_pivot.replace([np.inf, -np.inf], 0, inplace=True)
        
        arr_f_0=df_table_pivot.loc[df_table_pivot['TNACA_NAZWISKO']==tnaca]['FAKTURA_'+str(year1)].sum()
        arr_f_1=df_table_pivot.loc[df_table_pivot['TNACA_NAZWISKO']==tnaca]['FAKTURA_'+str(year2)].sum()
        try:
            arr_f_2=(arr_f_1-arr_f_0)/arr_f_0#*100
            arr_f_2=round(arr_f_2, 2)
        except:
            arr_f_2=0
        arr_i_0=df_table_pivot.loc[df_table_pivot['TNACA_NAZWISKO']==tnaca]['ILOSC_'+str(year1)].sum()
        arr_i_1=df_table_pivot.loc[df_table_pivot['TNACA_NAZWISKO']==tnaca]['ILOSC_'+str(year2)].sum()
        try:
            arr_i_2=(arr_i_1-arr_i_0)/arr_i_0#*100
            arr_i_2=round(arr_i_2, 2)
        except:
            arr_i_2=0
        arr_w_0=df_table_pivot.loc[df_table_pivot['TNACA_NAZWISKO']==tnaca]['WARTOSC_B_'+str(year1)].sum()
        arr_w_1=df_table_pivot.loc[df_table_pivot['TNACA_NAZWISKO']==tnaca]['WARTOSC_B_'+str(year2)].sum()
        try:
            arr_w_2=(arr_w_1-arr_w_0)/arr_w_0#*100
            arr_w_2=round(arr_w_2, 2)
        except:
            arr_w_2=0
        arr_k_0=df_table_pivot.loc[df_table_pivot['TNACA_NAZWISKO']==tnaca]['KOD_TEMA_'+str(year1)].sum()
        arr_k_1=df_table_pivot.loc[df_table_pivot['TNACA_NAZWISKO']==tnaca]['KOD_TEMA_'+str(year2)].sum()
        try:
            arr_k_2=(arr_k_1-arr_k_0)/arr_k_0#*100
            arr_k_2=round(arr_k_2, 2)
        except:
            arr_k_2=0
        arr_r_0=df_table_pivot.loc[df_table_pivot['TNACA_NAZWISKO']==tnaca]['ROZMIARSI_'+str(year1)].sum()
        arr_r_1=df_table_pivot.loc[df_table_pivot['TNACA_NAZWISKO']==tnaca]['ROZMIARSI_'+str(year2)].sum()
        try:
            arr_r_2=(arr_r_1-arr_r_0)/arr_r_0#*100
            arr_r_2=round(arr_r_2, 2)
        except:
            arr_r_2=0
        arr_day_0=df_table_pivot.loc[df_table_pivot['TNACA_NAZWISKO']==tnaca]['LICZBA_UNIKATOWYCH_DNI_'+str(year1)].sum()
        arr_day_1=df_table_pivot.loc[df_table_pivot['TNACA_NAZWISKO']==tnaca]['LICZBA_UNIKATOWYCH_DNI_'+str(year2)].sum()
        try:
            arr_day_2=(arr_day_1-arr_day_0)/arr_day_0#*100
            arr_day_2=round(arr_day_2, 2)
        except:
            arr_day_2=0
        # arr_minutes_0=df_table_pivot.loc[df_table_pivot['TNACA_NAZWISKO']==tnaca]['LICZBA_UNIKATOWYCH_MINUT_'+str(year1)].sum()
        # arr_minutes_1=df_table_pivot.loc[df_table_pivot['TNACA_NAZWISKO']==tnaca]['LICZBA_UNIKATOWYCH_MINUT_'+str(year2)].sum()
        # try:
        #     arr_minutes_2=(arr_minutes_1-arr_minutes_0)/arr_minutes_0#*100
        #     arr_minutes_2=round(arr_minutes_2, 2)
        # except:
        #     arr_minutes_2=0
        ###   
        # try:
        #     val_1_srednia_liczba_unikatowych_minut=round(arr_minutes_0/arr_day_0)
        # except:
        #     val_1_srednia_liczba_unikatowych_minut=0
        # try:
        #     val_2_srednia_liczba_unikatowych_minut=round(arr_minutes_1/arr_day_1)
        # except:
        #     val_2_srednia_liczba_unikatowych_minut=0
        # try:
        #     val_srednia_liczba_unikatowych_minut_dyn=round((val_2_srednia_liczba_unikatowych_minut-val_1_srednia_liczba_unikatowych_minut)/val_1_srednia_liczba_unikatowych_minut, 2)
        # except:
        #     val_srednia_liczba_unikatowych_minut_dyn=0
        ##
        #procent czasu pracy na kuponach do innych czynnosci
        ##
        try:
            val_1_proc_czasu_pracu=round(df_rcp_proc_time_total.loc[df_rcp_proc_time_total['TNACA_NAZWISKO']==tnaca]['procent_wykorzystania_czasu_pracy_na_kupony_'+str(year1)].values[0])
        except:
            val_1_proc_czasu_pracu=0
        try:
            val_2_proc_czasu_pracu=round(df_rcp_proc_time_total.loc[df_rcp_proc_time_total['TNACA_NAZWISKO']==tnaca]['procent_wykorzystania_czasu_pracy_na_kupony_'+str(year2)].values[0])
        except:
            val_2_proc_czasu_pracu=0
        try:
            val_proc_czasu_pracu_dyn=round((val_2_proc_czasu_pracu-val_1_proc_czasu_pracu)/val_1_proc_czasu_pracu, 2)
        except:
            val_proc_czasu_pracu_dyn=0
        ##
        #godziny pracy na calych kuponach 
        ##
        try:
            val_1_godziny_cale_kupony=round(df_rcp_time_cale_kupony_godziny_total.loc[df_rcp_time_cale_kupony_godziny_total['TNACA_NAZWISKO']==tnaca]['ile_minut_czynnosc_na_cale_kupony_'+str(year1)].values[0])
            val_1_godziny_cale_kupony=round(val_1_godziny_cale_kupony/60 , 2)
        except:
            val_1_godziny_cale_kupony=0
        try:
            val_2_godziny_cale_kupony=round(df_rcp_time_cale_kupony_godziny_total.loc[df_rcp_time_cale_kupony_godziny_total['TNACA_NAZWISKO']==tnaca]['ile_minut_czynnosc_na_cale_kupony_'+str(year2)].values[0])
            val_2_godziny_cale_kupony=round(val_2_godziny_cale_kupony/60 , 2)
        except:
            val_2_godziny_cale_kupony=0
        try:
            val_godziny_cale_kupony_dyn=round((val_2_godziny_cale_kupony-val_1_godziny_cale_kupony)/val_1_godziny_cale_kupony, 2)
        except:
            val_godziny_cale_kupony_dyn=0
        ##
        #godziny pracy na realizacji zamowien 
        ##
        try:
            val_1_godziny_real_zamowien=round(df_rcp_time_real_zamowienia_godziny_total.loc[df_rcp_time_real_zamowienia_godziny_total['TNACA_NAZWISKO']==tnaca]['ile_minut_czynnosc_na_realizacje_kupony_'+str(year1)].values[0])
            val_1_godziny_real_zamowien=round(val_1_godziny_real_zamowien/60 , 2)
        except:
            val_1_godziny_real_zamowien=0
        try:
            val_2_godziny_real_zamowien=round(df_rcp_time_real_zamowienia_godziny_total.loc[df_rcp_time_real_zamowienia_godziny_total['TNACA_NAZWISKO']==tnaca]['ile_minut_czynnosc_na_realizacje_kupony_'+str(year2)].values[0])
            val_2_godziny_real_zamowien=round(val_2_godziny_real_zamowien/60 , 2)
        except:
            val_2_godziny_real_zamowien=0
        try:
            val_godziny_real_zamowien_dyn=round((val_2_godziny_real_zamowien-val_1_godziny_real_zamowien)/val_1_godziny_real_zamowien, 2)
        except:
            val_godziny_real_zamowien_dyn=0
        df_table_pivot=df_table_pivot.append({'TNACA_NAZWISKO':tnaca,
                                               'month':'suma',
                                                'FAKTURA_'+str(year1):arr_f_0,
                                                'FAKTURA_'+str(year2):arr_f_1, 
                                                'FAKTURA_DYNAMIKA':arr_f_2,
                                                'ILOSC_'+str(year1):arr_i_0,
                                                'ILOSC_'+str(year2):arr_i_1, 
                                                'ILOSC_DYNAMIKA':arr_i_2,
                                                'ROZMIARSI_'+str(year1):arr_r_0,
                                                'ROZMIARSI_'+str(year2):arr_r_1, 
                                                'ROZMIARSI_DYNAMIKA':arr_r_2,
                                                'WARTOSC_B_'+str(year1):arr_w_0,
                                                'WARTOSC_B_'+str(year2):arr_w_1, 
                                                'WARTOSC_B_DYNAMIKA':arr_w_2,
                                                'KOD_TEMA_'+str(year1):arr_k_0,
                                                'KOD_TEMA_'+str(year2):arr_k_1, 
                                                'KOD_TEMA_DYNAMIKA':arr_k_2,
                                                 'PROCENT_CZASU_PRACY_NA_KUPONACH_'+str(year1):val_1_proc_czasu_pracu,
                                                  'PROCENT_CZASU_PRACY_NA_KUPONACH_'+str(year2):val_2_proc_czasu_pracu,
                                                  'PROCENT_CZASU_PRACY_NA_KUPONACH_DYNAMIKA':val_proc_czasu_pracu_dyn,
                                                     'LICZBA_GODZIN_NA_CALYCH_KUPONACH_'+str(year1):val_1_godziny_cale_kupony,
                                                      'LICZBA_GODZIN_NA_CALYCH_KUPONACH_'+str(year2):val_2_godziny_cale_kupony,
                                                  'LICZBA_GODZIN_NA_CALYCH_KUPONACH_DYNAMIKA':val_godziny_cale_kupony_dyn,
                                                     'LICZBA_GODZIN_NA_REAL_ZAMOWIEN_'+str(year1):val_1_godziny_real_zamowien,
                                                      'LICZBA_GODZIN_NA_REAL_ZAMOWIEN_'+str(year2):val_2_godziny_real_zamowien,
                                                  'LICZBA_GODZIN_NA_REAL_ZAMOWIEN_DYNAMIKA':val_godziny_real_zamowien_dyn, 
                                                'LICZBA_UNIKATOWYCH_DNI_'+str(year1):arr_day_0,
                                                'LICZBA_UNIKATOWYCH_DNI_'+str(year2):arr_day_1, 
                                                'LICZBA_UNIKATOWYCH_DNI_DYNAMIKA':arr_day_2},
                                             #    'LICZBA_UNIKATOWYCH_MINUT_'+str(year1):arr_minutes_0,
                                             #    'LICZBA_UNIKATOWYCH_MINUT_'+str(year2):arr_minutes_1, 
                                             #    'LICZBA_UNIKATOWYCH_MINUT_DYNAMIKA':arr_minutes_2,
                                             # 'SREDNIA_LICZBA_UNIKATOWYCH_MINUT_'+str(year1):val_1_srednia_liczba_unikatowych_minut,
                                             #  'SREDNIA_LICZBA_UNIKATOWYCH_MINUT_'+str(year2):val_2_srednia_liczba_unikatowych_minut,
                                             #  'SREDNIA_LICZBA_UNIKATOWYCH_MINUT_DYNAMIKA':val_srednia_liczba_unikatowych_minut_dyn},
                                              ignore_index=True)
        if czy_test=="TEST":
            break
        
        
    return df_table_pivot
    
    
    
    
    
def func_get_values_from_global_df(df, tnaca, year1, year2, mc, col_value, tnaca_czy_stol):
    if col_value=='FAKTURA':
        dict_with_agg={'FAKTURA':'nunique'}
    elif col_value=='ILOSC':
        dict_with_agg={'ILOSC':'sum'}
    elif col_value=='KOD_TEMA':
        dict_with_agg={'KOD_TEMA':'count'}
    elif col_value=='ROZMIARSI':
        dict_with_agg={'ROZMIARSI':'sum'}
    elif col_value=='WARTOSC_B':
        if tnaca_czy_stol=='tnaca':
            df=df.groupby(['Z_DNIA','year','month','TNACA1_NAZWISKO','TNACA2_NAZWISKO','FAKTURA']).agg({'WARTOSC_B':'first'}).reset_index()
            dict_with_agg={'WARTOSC_B':'sum'}
        else:
            df=df.groupby(['Z_DNIA','year','month','STOL_NAZWISKO','FAKTURA']).agg({'WARTOSC_B':'first'}).reset_index()
            dict_with_agg={'WARTOSC_B':'sum'}
    
    if tnaca_czy_stol=='tnaca':
        val_1=df.loc[df['year']==year1].loc[df['month']==mc].loc[(df['TNACA1_NAZWISKO']==tnaca) | (df['TNACA2_NAZWISKO']==tnaca)].agg(dict_with_agg).values[0]
        val_2=df.loc[df['year']==year2].loc[df['month']==mc].loc[(df['TNACA1_NAZWISKO']==tnaca) | (df['TNACA2_NAZWISKO']==tnaca)].agg(dict_with_agg).values[0]
    else:
        val_1=df.loc[df['year']==year1].loc[df['month']==mc].loc[(df['STOL_NAZWISKO']==tnaca)].agg(dict_with_agg).values[0]
        val_2=df.loc[df['year']==year2].loc[df['month']==mc].loc[(df['STOL_NAZWISKO']==tnaca)].agg(dict_with_agg).values[0]
    try:
        val_dyn=(val_2-val_1)/val_1#*100
    except:
        val_dyn=0
        
    val_1=round(val_1, 0)
    val_2=round(val_2, 0)
    val_dyn=round(val_dyn, 2)
    
    return val_1, val_2, val_dyn


def func_time_unique_day(df, tnaca, year1, year2, mc, tnaca_czy_stol):
    if tnaca_czy_stol=='tnaca':
        val_1=df.loc[df['year']==year1].loc[df['month']==mc].loc[(df['TNACA1_NAZWISKO']==tnaca) | (df['TNACA2_NAZWISKO']==tnaca)]['Z_DNIA'].nunique()
        val_2=df.loc[df['year']==year2].loc[df['month']==mc].loc[(df['TNACA1_NAZWISKO']==tnaca) | (df['TNACA2_NAZWISKO']==tnaca)]['Z_DNIA'].nunique()
    else:
        val_1=df.loc[df['year']==year1].loc[df['month']==mc].loc[(df['STOL_NAZWISKO']==tnaca)]['Z_DNIA'].nunique()
        val_2=df.loc[df['year']==year2].loc[df['month']==mc].loc[(df['STOL_NAZWISKO']==tnaca)]['Z_DNIA'].nunique()
        
    try:
        val_dyn=(val_2-val_1)/val_1#*100
    except:
        val_dyn=0
    val_1=round(val_1, 0)
    val_2=round(val_2, 0)
    val_dyn=round(val_dyn, 2)
    return val_1, val_2, val_dyn



def func_time_unique_minutes(df, tnaca, year1, year2, mc, tnaca_czy_stol):
    step=np.datetime64('2000-01-01 00:02:00')-np.datetime64('2000-01-01 00:01:00')
    
    if tnaca_czy_stol=='tnaca':
        df_tmp_1=df.loc[df['year']==year1].loc[df['month']==mc].loc[(df['TNACA1_NAZWISKO']==tnaca) | (df['TNACA2_NAZWISKO']==tnaca)]
        df_tmp_2=df.loc[df['year']==year2].loc[df['month']==mc].loc[(df['TNACA1_NAZWISKO']==tnaca) | (df['TNACA2_NAZWISKO']==tnaca)]
    else:
        df_tmp_1=df.loc[df['year']==year1].loc[df['month']==mc].loc[(df['STOL_NAZWISKO']==tnaca)]
        df_tmp_2=df.loc[df['year']==year2].loc[df['month']==mc].loc[(df['STOL_NAZWISKO']==tnaca)]
    
    val_time_day_unique_1=func_time_unique_minutes_calculation(df_tmp_1, step)
    val_time_day_unique_2=func_time_unique_minutes_calculation(df_tmp_2, step)
    val_dyn=(val_time_day_unique_2-val_time_day_unique_1)/val_time_day_unique_1#*100
    try:
        val_dyn=(val_time_day_unique_2-val_time_day_unique_1)/val_time_day_unique_1#*100
    except:
        val_dyn=0
    val_time_day_unique_1=round(val_time_day_unique_1, 0)
    val_time_day_unique_2=round(val_time_day_unique_2, 0)
    val_dyn=round(val_dyn, 2)
    
    return val_time_day_unique_1, val_time_day_unique_2, val_dyn
    
    
def func_time_unique_minutes_calculation(df, step):
    arr_time_day_unique=[]
    for day in df['Z_DNIA'].unique():
        arr_time_day=np.array([], dtype='datetime64')
        
        for i in df.loc[df['Z_DNIA']==day].index:
            arr_tmp=np.arange(df.loc[i,'date_start'],
                              df.loc[i,'date_finish'],
                             step)
            arr_time_day=np.append(arr_time_day, arr_tmp)
        arr_time_day_unique=np.append(arr_time_day_unique, [len(np.unique(arr_time_day))])
        
    return np.sum(arr_time_day_unique)




def func_append_dfs_with_typ_wartosc_b(df_m, df_v):
    arr_columns_with_index=['zmiana', 'month']
    arr_columns_to_rename=['WARTOSC_B_2021', 'WARTOSC_B_2022', 'WARTOSC_B_DYNAMIKA',
                          'LICZBA_UNIKATOWYCH_DNI_2021', 'LICZBA_UNIKATOWYCH_DNI_2022',
                           'LICZBA_UNIKATOWYCH_DNI_DYNAMIKA']#, 'LICZBA_UNIKATOWYCH_MINUT_2021',
                           # 'LICZBA_UNIKATOWYCH_MINUT_2022', 'LICZBA_UNIKATOWYCH_MINUT_DYNAMIKA',
                           # 'SREDNIA_LICZBA_UNIKATOWYCH_MINUT_2021',
                           # 'SREDNIA_LICZBA_UNIKATOWYCH_MINUT_2022',
                           # 'SREDNIA_LICZBA_UNIKATOWYCH_MINUT_DYNAMIKA']
    dict_rename_m={}
    dict_rename_v={}
    for col in arr_columns_to_rename:
        dict_rename_m[col]=col+'_M'
        dict_rename_v[col]=col+'_V'
    
    df_m=df_m.rename(columns=dict_rename_m)
    df_v=df_v.rename(columns=dict_rename_v)
    
    df=pd.concat([df_m, df_v]).groupby(arr_columns_with_index).sum().reset_index()
    
    
    df['WARTOSC_B_suma_2021']=df['WARTOSC_B_2021_M']+df['WARTOSC_B_2021_V']
    df['WARTOSC_B_suma_2022']=df['WARTOSC_B_2022_M']+df['WARTOSC_B_2022_V']
    df['WARTOSC_B_2021_udzial_M']=round(df['WARTOSC_B_2021_M']/df['WARTOSC_B_suma_2021'], 2)
    df['WARTOSC_B_2022_udzial_M']=round(df['WARTOSC_B_2022_M']/df['WARTOSC_B_suma_2022'], 2)
    df['WARTOSC_B_2021_udzial_V']=round(df['WARTOSC_B_2021_V']/df['WARTOSC_B_suma_2021'], 2)
    df['WARTOSC_B_2022_udzial_V']=round(df['WARTOSC_B_2022_V']/df['WARTOSC_B_suma_2022'], 2)
    
    
    
    for i_name in ['FAKTURA', 'ILOSC', 'KOD_TEMA', 'ROZMIARSI']:
        df[i_name+'_DYNAMIKA']=round((df[i_name+'_2022']-df[i_name+'_2021'])/df[i_name+'_2021'],2)

        
    df=df[['zmiana', 'month', 'FAKTURA_2021', 'FAKTURA_2022', 'FAKTURA_DYNAMIKA',
           'ILOSC_2021', 'ILOSC_2022', 'ILOSC_DYNAMIKA', 
           'WARTOSC_B_suma_2021',
           'WARTOSC_B_2021_M',
           'WARTOSC_B_2021_udzial_M',
           'WARTOSC_B_2021_V',
           'WARTOSC_B_2021_udzial_V',
           'WARTOSC_B_suma_2022',
           'WARTOSC_B_2022_M',
           'WARTOSC_B_2022_udzial_M',
           'WARTOSC_B_2022_V',
           'WARTOSC_B_2022_udzial_V',
           'WARTOSC_B_DYNAMIKA_M', 
            'WARTOSC_B_DYNAMIKA_V',
           'KOD_TEMA_2021',
           'KOD_TEMA_2022', 'KOD_TEMA_DYNAMIKA',
          'ROZMIARSI_2021',
           'ROZMIARSI_2022', 'ROZMIARSI_DYNAMIKA']]
    
    return df


def func_append_dfs_with_typ_wartosc_b_suma_sum(df):
    
    dict_agg={}
    for i in df.columns:
        if (i=='month') or ('DYNAMIKA' in i) or (i=='zmiana') or ('udzial' in i):
            continue
        else:
            dict_agg[i]='sum'
            
    df=df.loc[df['month']=='suma'].groupby(['month']).agg(dict_agg).reset_index()
    for i_name in ['FAKTURA', 'ILOSC', 'KOD_TEMA', 'ROZMIARSI']:
        df[i_name+'_DYNAMIKA']=round((df[i_name+'_2022']-df[i_name+'_2021'])/df[i_name+'_2021'],2)
    for i_name in ['WARTOSC_B_{}_M', 'WARTOSC_B_{}_V']:
        df[i_name.format('DYNAMIKA')]=round((df[i_name.format('2022')]-df[i_name.format('2021')])/df[i_name.format('2021')],2)
        
    df['WARTOSC_B_2021_udzial_M']=round(df['WARTOSC_B_2021_M']/df['WARTOSC_B_suma_2021'], 2)
    df['WARTOSC_B_2022_udzial_M']=round(df['WARTOSC_B_2022_M']/df['WARTOSC_B_suma_2022'], 2)
    df['WARTOSC_B_2021_udzial_V']=round(df['WARTOSC_B_2021_V']/df['WARTOSC_B_suma_2021'], 2)
    df['WARTOSC_B_2022_udzial_V']=round(df['WARTOSC_B_2022_V']/df['WARTOSC_B_suma_2022'], 2)
    df['zmiana']='pierwsza+druga'
    return df

In [121]:
def func_create_pivot_table_per_hour(df, year1, year2, col_with_group, col_to_div, col_with_time):
    df=df[col_with_group + col_to_div + col_with_time]
    for div in col_to_div:
        if str(year1) in div:
            df[div+'_na_godzine_realizacji_zamowien']=round(df[div]/df[col_with_time[3]], 2)
        elif str(year2) in div:
            df[div+'_na_godzine_realizacji_zamowien']=round(df[div]/df[col_with_time[4]], 2)
        else:
            continue
    df=df.fillna(0)
    df.replace([np.inf, -np.inf], 0, inplace=True)    
    
    return df
    

## calc table - wywolania

In [122]:
"""KOD
        -lista miesięcy do raportu
    """
lista_mc_do_tabeli=[1,2,3,4,5,6,7,8,9]

## usunac osoby na sumach z zerami!!!!
## zmienic z cm na metry w rozmiarsi!!!

In [123]:
"""FUNKCJA
        -sprawdza czy porownawczy raport zawiera takie same dane
        -dla nazwiska S. KAPAŁA
        -jeśli chcemy zmienić lub dodać osobę musimy zmienić także w funckji wyliczającą raport
        -jeśli w raportach nie ma różnic zwracana jest prawda
    """
def func_check_correct(df_new, lista_mc_do_tabeli, path):
    df_old=pd.read_excel(path).drop('Unnamed: 0', axis=1)
    df_old=df_old.loc[df_old['TNACA_NAZWISKO']=='S. KAPAŁA']
    df_old=df_old.loc[df_old['month'].isin(lista_mc_do_tabeli[:-2])].reset_index().drop('index', axis=1)
    df_new=df_new.loc[df_new['month'].isin(lista_mc_do_tabeli[:-2])]
    #display(df_old)
    #display(df_new)
    if df_old.compare(df_new, keep_shape=False, keep_equal=False).empty == True:
        return True
    else:
        return False

In [124]:
# aa=func_read_list_of_names(r'C:\Users\kgrzebien\Desktop\python_tasks\kupony'+'\\fast_kpi')
# arr_ind_name_of_raport=[]
# arr_ind_name_of_raport_per_hour=[]

# for i, element in enumerate(aa):
#     if 'df_pivot_table_agata_with_time_sumy_procent' in element:
#         if 'per_hour' in element:
#             arr_ind_name_of_raport_per_hour=np.append(arr_ind_name_of_raport_per_hour, int(i))
#         else:
#             arr_ind_name_of_raport=np.append(arr_ind_name_of_raport, int(i))

In [125]:
"""FUNKCJA
        -podaje date ostatniego dnia w danych żeby dodać do nazwy plików
    """
def func_name_raport(df):
    name_of_raport='df_pivot_table_agata_with_time_sumy_procent_'
    name_of_raport_per_hour='df_pivot_table_agata_with_time_sumy_procent_per_hour_'
    str_max_date=str(df['Z_DNIA'].max())
    date_to_add=str_max_date[2:4]+'_'+str_max_date[5:7]+'_'+str_max_date[8:10]
    name_of_raport+=date_to_add+'.xlsx'
    name_of_raport_per_hour+=date_to_add+'.xlsx'
    return name_of_raport, name_of_raport_per_hour

In [126]:
"""FUNKCJA
        -usunięcie zer
        -wyliczenia na godzine
        -zapis raportów do plików
    """
def func_save(df_pivot_table_agata,
              df):
    name_of_raport, name_of_raport_per_hour=func_name_raport(df)
    
    df_pivot_table_agata_bez_zer=df_pivot_table_agata.loc[df_pivot_table_agata['TNACA_NAZWISKO'].isin(df_pivot_table_agata.loc[df_pivot_table_agata['month']=='suma'].\
                                                                                                                        loc[(df_pivot_table_agata['FAKTURA_2022']!=0) |\
                                                                                                                            (df_pivot_table_agata['FAKTURA_2021']!=0)]['TNACA_NAZWISKO'].unique())]
    
    df_pivot_table_agata_bez_zer.to_excel(path_to_catalog+name_of_raport)
    
    
    df_pivot_table_agata_per_hour=func_create_pivot_table_per_hour(df_pivot_table_agata_bez_zer,
                                                     2021, 2022, 
                                                     ['TNACA_NAZWISKO','month'], 
                                                     ['ROZMIARSI_2021','ROZMIARSI_2022','ROZMIARSI_DYNAMIKA','KOD_TEMA_2021','KOD_TEMA_2022','KOD_TEMA_DYNAMIKA'],
                                                     ['LICZBA_GODZIN_NA_CALYCH_KUPONACH_2021','LICZBA_GODZIN_NA_CALYCH_KUPONACH_2022','LICZBA_GODZIN_NA_CALYCH_KUPONACH_DYNAMIKA',
                                                      'LICZBA_GODZIN_NA_REAL_ZAMOWIEN_2021','LICZBA_GODZIN_NA_REAL_ZAMOWIEN_2022','LICZBA_GODZIN_NA_REAL_ZAMOWIEN_DYNAMIKA'])
    
    df_pivot_table_agata_per_hour.to_excel(path_to_catalog+name_of_raport_per_hour)
    
    
    return

In [127]:
"""FUNKCJA
        -uruchamia test, kiedy w argumencie jest "TEST"
        -uruchamia sprawdzenie porównania
        -uruchamia wyliczenie całego raportu
        -uruchamia zapis do plików
    """
def func_run_calc_table_test(df):
    df_pivot_table_agata=func_create_pivot_table_agata(df,#.loc[df['max_dzien_pracy_tnaca1']>=np.datetime64('2022-01-01')],
                                                   2021, 2022,
                                                   lista_mc_do_tabeli,
                                                   'tnaca',
                                                  'TEST')
    flag_correct=func_check_correct(df_pivot_table_agata,
                                       lista_mc_do_tabeli,
                                       path_to_catalog+'df_pivot_table_agata_with_time_sumy_procent_22_08.xlsx') #podaj nazwe pliku do ktorego sie porownujemy
    if flag_correct:
        df_pivot_table_agata=func_create_pivot_table_agata(df,#.loc[df['max_dzien_pracy_tnaca1']>=np.datetime64('2022-01-01')],
                                                   2021, 2022,
                                                   lista_mc_do_tabeli,
                                                   'tnaca',
                                                  'ALL')
        func_save(df_pivot_table_agata,
                 df)
        print('PLIKI ZAPISANE!')
    else:
        print('RÓŻNICE W RAPORTACH!!')

In [128]:
"""KOD
        -uruchamia funkcje wyliczającą raporty
    """
func_run_calc_table_test(df)

100%|████████████████████████████████████████████████████████████████████████████████| 103/103 [16:27<00:00,  9.59s/it]


PLIKI ZAPISANE!


In [134]:
display(df_rcp.loc[df_rcp['KOD_OPER']==1918].loc[df_rcp['DATA']>=np.datetime64('2022-09-01')]) #stasica
display(df_rcp.loc[df_rcp['KOD_OPER']==438].loc[df_rcp['DATA']>=np.datetime64('2022-09-01')]) #
display(df_rcp.loc[df_rcp['KOD_OPER']==267].loc[df_rcp['DATA']>=np.datetime64('2022-09-01')]) #tyc
display(df_rcp.loc[df_rcp['KOD_OPER']==345].loc[df_rcp['DATA']>=np.datetime64('2022-09-01')])  #rypien

,ID_ZAD,DATA,ROZ_DNIA,ZAK_DNIA,GODZ_PL,GODZ_ROZ,GODZ_ZAK,KOD_OPER,date_start,date_finish,ZADANIE,zadanie_group
94023,ZP0000000007,2022-09-01,,,,18:41:07,19:00:36,1918,2022-09-01 18:41:00,2022-09-01 19:00:00,PRZERWA,inne
94026,ZP0000000994,2022-09-01,,,,19:00:36,20:11:57,1918,2022-09-01 19:00:00,2022-09-01 20:11:00,KUPONY-REALIZACJA ZAMÓWIENIA,KUPONY-REALIZACJA ZAMÓWIENIA
94027,ZP0000000996,2022-09-01,,,,20:11:57,20:34:32,1918,2022-09-01 20:11:00,2022-09-01 20:34:00,KUPONY -PRACE DODATKOWE,kupony_inne
94029,ZP0000000994,2022-09-01,,,,20:34:32,21:32:54,1918,2022-09-01 20:34:00,2022-09-01 21:32:00,KUPONY-REALIZACJA ZAMÓWIENIA,KUPONY-REALIZACJA ZAMÓWIENIA
94036,ZP0000000996,2022-09-01,,,,21:32:54,21:59:49,1918,2022-09-01 21:32:00,2022-09-01 21:59:00,KUPONY -PRACE DODATKOWE,kupony_inne
94203,ZP0000000994,2022-09-01,,,,13:38:38,18:41:07,1918,2022-09-01 13:38:00,2022-09-01 18:41:00,KUPONY-REALIZACJA ZAMÓWIENIA,KUPONY-REALIZACJA ZAMÓWIENIA
94469,ZP0000000036,2022-09-02,,,,07:20:18,15:59:19,1918,2022-09-02 07:20:00,2022-09-02 15:59:00,SPOTKANIE-OBSŁUGA,inne
94600,ZP0000000036,2022-09-05,,,,06:33:11,16:04:36,1918,2022-09-05 06:33:00,2022-09-05 16:04:00,SPOTKANIE-OBSŁUGA,inne


,ID_ZAD,DATA,ROZ_DNIA,ZAK_DNIA,GODZ_PL,GODZ_ROZ,GODZ_ZAK,KOD_OPER,date_start,date_finish,ZADANIE,zadanie_group
94050,ZP0000000994,2022-09-01,,,,06:10:47,09:44:12,438,2022-09-01 06:10:00,2022-09-01 09:44:00,KUPONY-REALIZACJA ZAMÓWIENIA,KUPONY-REALIZACJA ZAMÓWIENIA
94128,ZP0000000996,2022-09-01,,,,09:44:12,10:29:45,438,2022-09-01 09:44:00,2022-09-01 10:29:00,KUPONY -PRACE DODATKOWE,kupony_inne
94130,ZP0000000007,2022-09-01,,,,10:29:45,10:49:32,438,2022-09-01 10:29:00,2022-09-01 10:49:00,PRZERWA,inne
94134,ZP0000000996,2022-09-01,,,,10:49:32,13:30:25,438,2022-09-01 10:49:00,2022-09-01 13:30:00,KUPONY -PRACE DODATKOWE,kupony_inne
94195,ZP0000000996,2022-09-01,,,,05:26:21,06:10:47,438,2022-09-01 05:26:00,2022-09-01 06:10:00,KUPONY -PRACE DODATKOWE,kupony_inne
94566,ZP0000000007,2022-09-02,,,,10:30:40,10:50:57,438,2022-09-02 10:30:00,2022-09-02 10:50:00,PRZERWA,inne
94571,ZP0000000996,2022-09-02,,,,10:50:57,13:30:20,438,2022-09-02 10:50:00,2022-09-02 13:30:00,KUPONY -PRACE DODATKOWE,kupony_inne
94710,ZP0000000996,2022-09-02,,,,05:28:25,10:30:40,438,2022-09-02 05:28:00,2022-09-02 10:30:00,KUPONY -PRACE DODATKOWE,kupony_inne


,ID_ZAD,DATA,ROZ_DNIA,ZAK_DNIA,GODZ_PL,GODZ_ROZ,GODZ_ZAK,KOD_OPER,date_start,date_finish,ZADANIE,zadanie_group
94201,ZP0000000996,2022-09-01,,,,13:53:38,14:25:06,267,2022-09-01 13:53:00,2022-09-01 14:25:00,KUPONY -PRACE DODATKOWE,kupony_inne
94369,ZP0000000994,2022-09-01,,,,14:25:06,18:01:31,267,2022-09-01 14:25:00,2022-09-01 18:01:00,KUPONY-REALIZACJA ZAMÓWIENIA,KUPONY-REALIZACJA ZAMÓWIENIA
94381,ZP0000000007,2022-09-01,,,,18:01:31,18:20:05,267,2022-09-01 18:01:00,2022-09-01 18:20:00,PRZERWA,inne
94385,ZP0000000994,2022-09-01,,,,18:20:05,19:41:32,267,2022-09-01 18:20:00,2022-09-01 19:41:00,KUPONY-REALIZACJA ZAMÓWIENIA,KUPONY-REALIZACJA ZAMÓWIENIA
94391,ZP0000000996,2022-09-01,,,,19:41:32,19:57:31,267,2022-09-01 19:41:00,2022-09-01 19:57:00,KUPONY -PRACE DODATKOWE,kupony_inne
94392,ZP0000000994,2022-09-01,,,,19:57:31,21:31:00,267,2022-09-01 19:57:00,2022-09-01 21:31:00,KUPONY-REALIZACJA ZAMÓWIENIA,KUPONY-REALIZACJA ZAMÓWIENIA
94395,ZP0000000996,2022-09-01,,,,21:31:00,21:59:29,267,2022-09-01 21:31:00,2022-09-01 21:59:00,KUPONY -PRACE DODATKOWE,kupony_inne
94630,ZP0000000994,2022-09-02,,,,15:04:51,18:00:40,267,2022-09-02 15:04:00,2022-09-02 18:00:00,KUPONY-REALIZACJA ZAMÓWIENIA,KUPONY-REALIZACJA ZAMÓWIENIA
94635,ZP0000000007,2022-09-02,,,,18:00:40,18:17:53,267,2022-09-02 18:00:00,2022-09-02 18:17:00,PRZERWA,inne
94637,ZP0000000994,2022-09-02,,,,18:17:53,21:59:43,267,2022-09-02 18:17:00,2022-09-02 21:59:00,KUPONY-REALIZACJA ZAMÓWIENIA,KUPONY-REALIZACJA ZAMÓWIENIA


,ID_ZAD,DATA,ROZ_DNIA,ZAK_DNIA,GODZ_PL,GODZ_ROZ,GODZ_ZAK,KOD_OPER,date_start,date_finish,ZADANIE,zadanie_group
94061,ZP0000000996,2022-09-01,,,,13:49:28,13:56:56,345,2022-09-01 13:49:00,2022-09-01 13:56:00,KUPONY -PRACE DODATKOWE,kupony_inne
94368,ZP0000000994,2022-09-01,,,,13:56:56,18:01:26,345,2022-09-01 13:56:00,2022-09-01 18:01:00,KUPONY-REALIZACJA ZAMÓWIENIA,KUPONY-REALIZACJA ZAMÓWIENIA
94450,ZP0000000007,2022-09-01,,,,18:01:26,18:20:07,345,2022-09-01 18:01:00,2022-09-01 18:20:00,PRZERWA,inne
94451,ZP0000000994,2022-09-01,,,,18:20:07,21:28:30,345,2022-09-01 18:20:00,2022-09-01 21:28:00,KUPONY-REALIZACJA ZAMÓWIENIA,KUPONY-REALIZACJA ZAMÓWIENIA
94452,ZP0000000996,2022-09-01,,,,21:28:30,21:59:45,345,2022-09-01 21:28:00,2022-09-01 21:59:00,KUPONY -PRACE DODATKOWE,kupony_inne
94603,ZP0000000994,2022-09-05,,,,05:21:46,10:59:56,345,2022-09-05 05:21:00,2022-09-05 10:59:00,KUPONY-REALIZACJA ZAMÓWIENIA,KUPONY-REALIZACJA ZAMÓWIENIA
94605,ZP0000000007,2022-09-05,,,,10:59:56,11:18:56,345,2022-09-05 10:59:00,2022-09-05 11:18:00,PRZERWA,inne
94608,ZP0000000994,2022-09-05,,,,11:18:56,13:04:47,345,2022-09-05 11:18:00,2022-09-05 13:04:00,KUPONY-REALIZACJA ZAMÓWIENIA,KUPONY-REALIZACJA ZAMÓWIENIA
94611,ZP0000000996,2022-09-05,,,,13:04:47,13:19:28,345,2022-09-05 13:04:00,2022-09-05 13:19:00,KUPONY -PRACE DODATKOWE,kupony_inne
94613,ZP0000000994,2022-09-05,,,,13:19:28,13:20:13,345,2022-09-05 13:19:00,2022-09-05 13:20:00,KUPONY-REALIZACJA ZAMÓWIENIA,KUPONY-REALIZACJA ZAMÓWIENIA


In [137]:
func_create_pivot_table_agata(df,#.loc[df['max_dzien_pracy_tnaca1']>=np.datetime64('2022-01-01')],
                                                   2021, 2022,
                                                   lista_mc_do_tabeli,
                                                   'tnaca',
                                                  'TEST')

  0%|                                                                                          | 0/103 [00:09<?, ?it/s]


,TNACA_NAZWISKO,month,FAKTURA_2021,FAKTURA_2022,FAKTURA_DYNAMIKA,ILOSC_2021,ILOSC_2022,ILOSC_DYNAMIKA,ROZMIARSI_2021,ROZMIARSI_2022,ROZMIARSI_DYNAMIKA,WARTOSC_B_2021,WARTOSC_B_2022,WARTOSC_B_DYNAMIKA,KOD_TEMA_2021,KOD_TEMA_2022,KOD_TEMA_DYNAMIKA,PROCENT_CZASU_PRACY_NA_KUPONACH_2021,PROCENT_CZASU_PRACY_NA_KUPONACH_2022,PROCENT_CZASU_PRACY_NA_KUPONACH_DYNAMIKA,LICZBA_GODZIN_NA_CALYCH_KUPONACH_2021,LICZBA_GODZIN_NA_CALYCH_KUPONACH_2022,LICZBA_GODZIN_NA_CALYCH_KUPONACH_DYNAMIKA,LICZBA_GODZIN_NA_REAL_ZAMOWIEN_2021,LICZBA_GODZIN_NA_REAL_ZAMOWIEN_2022,LICZBA_GODZIN_NA_REAL_ZAMOWIEN_DYNAMIKA,LICZBA_UNIKATOWYCH_DNI_2021,LICZBA_UNIKATOWYCH_DNI_2022,LICZBA_UNIKATOWYCH_DNI_DYNAMIKA
0,K. RYPIEŃ,1,0,385,0.00,0.0,4540.0,0.00,0,1849,0.00,0.0,215782.0,0.00,0,726,0.00,0,94,0.00,0.00,140.87,0.00,0.00,110.72,0.00,0,18,0.00
1,K. RYPIEŃ,2,0,359,0.00,0.0,4097.0,0.00,0,1935,0.00,0.0,175525.0,0.00,0,739,0.00,0,78,0.00,0.00,114.62,0.00,0.00,100.60,0.00,0,17,0.00
2,K. RYPIEŃ,3,0,427,0.00,0.0,5162.0,0.00,0,2447,0.00,0.0,242018.0,0.00,0,941,0.00,0,98,0.00,0.00,145.18,0.00,0.00,124.50,0.00,0,18,0.00
3,K. RYPIEŃ,4,0,384,0.00,0.0,4076.0,0.00,0,2059,0.00,0.0,168480.0,0.00,0,773,0.00,0,88,0.00,0.00,135.83,0.00,0.00,103.13,0.00,0,17,0.00
4,K. RYPIEŃ,5,0,378,0.00,0.0,4698.0,0.00,0,2058,0.00,0.0,198710.0,0.00,0,784,0.00,0,89,0.00,0.00,148.93,0.00,0.00,92.85,0.00,0,18,0.00
5,K. RYPIEŃ,6,0,401,0.00,0.0,4907.0,0.00,0,2063,0.00,0.0,210560.0,0.00,0,789,0.00,0,94,0.00,0.00,130.87,0.00,0.00,108.88,0.00,0,17,0.00
6,K. RYPIEŃ,7,0,206,0.00,0.0,2252.0,0.00,0,1002,0.00,0.0,96972.0,0.00,0,391,0.00,0,96,0.00,0.00,85.52,0.00,0.00,57.80,0.00,0,10,0.00
7,K. RYPIEŃ,8,247,341,0.38,2179.0,3616.0,0.66,957,1646,0.72,89008.0,151953.0,0.71,373,620,0.66,97,100,0.03,71.32,181.00,1.54,54.72,132.63,1.42,9,21,1.33
8,K. RYPIEŃ,9,511,58,-0.89,5282.0,786.0,-0.85,2456,351,-0.86,227958.0,34512.0,-0.85,959,137,-0.86,96,100,0.04,173.88,24.57,-0.86,137.90,22.55,-0.84,22,3,-0.86
9,K. RYPIEŃ,suma,758,2939,2.88,7461.0,34134.0,3.57,3413,15410,3.52,316966.0,1494512.0,3.72,1332,5900,3.43,97,92,-0.05,245.20,1107.38,3.52,192.62,853.67,3.43,31,139,3.48


In [139]:
df_with_minutes_rcp.loc[df_with_minutes_rcp['KOD_OPER']==345].loc[df_with_minutes_rcp['DATA']>=np.datetime64('2022-09-01')]

,KOD_OPER,DATA,ile_minut_czynnosc,ile_minut_dzien,procent_wykorzystania,zadanie,zadanie_group,year,month,year_month,week,year_week,zadania_ogolnie_5_grupy,zadania_ogolnie_3_grupy,zadania_ogolnie_4_grupy
2324,345,2022-09-01 00:00:00,7,490.0,1.428571,KUPONY -PRACE DODATKOWE,kupony_inne,2022,9,2022-09-01,35,2022--35,1 - kupony_inne,1-kupony,1-kupony
2325,345,2022-09-01 00:00:00,245,490.0,50.0,KUPONY-REALIZACJA ZAMÓWIENIA,KUPONY-REALIZACJA ZAMÓWIENIA,2022,9,2022-09-01,35,2022--35,1 - KUPONY-REALIZACJA ZAMÓWIENIA,1-kupony,1-kupony
2326,345,2022-09-01 00:00:00,19,490.0,3.877551,PRZERWA,inne,2022,9,2022-09-01,35,2022--35,1 - kupony_inne,1-kupony,1-kupony
2327,345,2022-09-01 00:00:00,188,490.0,38.367347,KUPONY-REALIZACJA ZAMÓWIENIA,KUPONY-REALIZACJA ZAMÓWIENIA,2022,9,2022-09-01,35,2022--35,1 - KUPONY-REALIZACJA ZAMÓWIENIA,1-kupony,1-kupony
2328,345,2022-09-01 00:00:00,31,490.0,6.326531,KUPONY -PRACE DODATKOWE,kupony_inne,2022,9,2022-09-01,35,2022--35,1 - kupony_inne,1-kupony,1-kupony
2340,345,2022-09-05 00:00:00,338,490.0,68.979592,KUPONY-REALIZACJA ZAMÓWIENIA,KUPONY-REALIZACJA ZAMÓWIENIA,2022,9,2022-09-01,36,2022--36,1 - KUPONY-REALIZACJA ZAMÓWIENIA,1-kupony,1-kupony
2341,345,2022-09-05 00:00:00,19,490.0,3.877551,PRZERWA,inne,2022,9,2022-09-01,36,2022--36,1 - kupony_inne,1-kupony,1-kupony
2342,345,2022-09-05 00:00:00,106,490.0,21.632653,KUPONY-REALIZACJA ZAMÓWIENIA,KUPONY-REALIZACJA ZAMÓWIENIA,2022,9,2022-09-01,36,2022--36,1 - KUPONY-REALIZACJA ZAMÓWIENIA,1-kupony,1-kupony
2343,345,2022-09-05 00:00:00,15,490.0,3.061224,KUPONY -PRACE DODATKOWE,kupony_inne,2022,9,2022-09-01,36,2022--36,1 - kupony_inne,1-kupony,1-kupony
2344,345,2022-09-05 00:00:00,1,490.0,0.204082,KUPONY-REALIZACJA ZAMÓWIENIA,KUPONY-REALIZACJA ZAMÓWIENIA,2022,9,2022-09-01,36,2022--36,1 - KUPONY-REALIZACJA ZAMÓWIENIA,1-kupony,1-kupony


In [ ]:
# df_pivot_table_agata_bez_zer=df_pivot_table_agata.loc[df_pivot_table_agata['TNACA_NAZWISKO'].isin(df_pivot_table_agata.loc[df_pivot_table_agata['month']=='suma'].\
#                                                                                                                         loc[(df_pivot_table_agata['FAKTURA_2022']!=0) |\
#                                                                                                                             (df_pivot_table_agata['FAKTURA_2021']!=0)]['TNACA_NAZWISKO'].unique())]

In [ ]:
# df_pivot_table_agata_per_hour=func_create_pivot_table_per_hour(df_pivot_table_agata_bez_zer,
#                                                      2021, 2022, 
#                                                      ['TNACA_NAZWISKO','month'], 
#                                                      ['ROZMIARSI_2021','ROZMIARSI_2022','ROZMIARSI_DYNAMIKA','KOD_TEMA_2021','KOD_TEMA_2022','KOD_TEMA_DYNAMIKA'],
#                                                      ['LICZBA_GODZIN_NA_CALYCH_KUPONACH_2021','LICZBA_GODZIN_NA_CALYCH_KUPONACH_2022','LICZBA_GODZIN_NA_CALYCH_KUPONACH_DYNAMIKA',
#                                                       'LICZBA_GODZIN_NA_REAL_ZAMOWIEN_2021','LICZBA_GODZIN_NA_REAL_ZAMOWIEN_2022','LICZBA_GODZIN_NA_REAL_ZAMOWIEN_DYNAMIKA'])

In [ ]:
# df_pivot_table_agata_bez_zer.to_excel(path_to_catalog+'df_pivot_table_agata_with_time_sumy_procent_22_08.xlsx')

In [ ]:
# df_pivot_table_agata_per_hour.to_excel(path_to_catalog+'df_pivot_table_agata_with_time_sumy_procent_per_hour_22_08.xlsx')